In [82]:
import pandas as pd
import numpy as np

from utils.feature_engineering import (
    # parse_and_convert_numeric_cols,
    calculate_experience_match_score,
    calculate_salary_fit_score,
    prepare_nlp_text_columns,
    calculate_nlp_similarity_st, # Function for Sentence Transformers
    calculate_nlp_similarity_tfidf, # Function for TF-IDF
    calculate_geo_features,
    calculate_overall_score,
    drop_columns_except,
    SentenceTransformer, # Import SentenceTransformer to load model
    TfidfVectorizer, # Import TfidfVectorizer to fit vectorizer
    cosine_similarity # Import cosine_similarity if needed outside functions (optional)
)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier # Or other model that handles NaNs
from sklearn.metrics import accuracy_score, classification_report, f1_score


from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import time # To handle geocoding rate limits
import warnings # To manage potential warnings

# Ignore warnings for cleaner output, but be cautious in real applications
warnings.filterwarnings('ignore')

### Load the data

In [83]:
import pandas as pd 
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.width', None)         # Don't wrap lines
# pd.set_option('display.max_colwidth', None)  # Don't truncate column contents
df = pd.read_excel('Dataset_2.0_Akkodis.xlsx')

df

,ID,Candidate State,Age Range,Residence,Sex,Protected category,TAG,Study area,Study Title,Years Experience,Sector,Last Role,Year of insertion,Year of Recruitment,Recruitment Request,Assumption Headquarters,Job Family Hiring,Job Title Hiring,event_type__val,event_feedback,linked_search__key,Overall,Job Description,Candidate Profile,Years Experience.1,Minimum Ral,Ral Maximum,Study Level,Study Area.1,Akkodis headquarters,Current Ral,Expected Ral,Technical Skills,Standing/Position,Comunication,Maturity,Dynamism,Mobility,English
0,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],Automotive,Diagnostic/Test engineer,[2018],[2021],E/E Diagnostic Integration Engineer - Automotive,Milan,Engineering,Consultant,Candidate notification,NaN,NaN,NaN,"The candidate, inserted within a multidiscipli...",The ideal candidate has a degree in Electronic...,[1-3],26-28K,30-32K,Five-year degree,electronic Engineering,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],Automotive,Diagnostic/Test engineer,[2018],[2021],E/E Diagnostic Integration Engineer - Automotive,Milan,Engineering,Consultant,BM interview,NaN,RS18.0145,NaN,"The candidate, inserted within a multidiscipli...",The ideal candidate has a degree in Electronic...,[1-3],26-28K,30-32K,Five-year degree,electronic Engineering,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],Automotive,Diagnostic/Test engineer,[2018],[2021],E/E Diagnostic Integration Engineer - Automotive,Milan,Engineering,Consultant,Contact note,NaN,NaN,NaN,"The candidate, inserted within a multidiscipli...",The ideal candidate has a degree in Electronic...,[1-3],26-28K,30-32K,Five-year degree,electronic Engineering,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],Automotive,Diagnostic/Test engineer,[2018],[2021],E/E Diagnostic Integration Engineer - Automotive,Milan,Engineering,Consultant,BM interview,OK,RS18.0114,~ 2 - Medium,"The candidate, inserted within a multidiscipli...",The ideal candidate has a degree in Electronic...,[1-3],26-28K,30-32K,Five-year degree,electronic Engineering,Modena,22-24 K,24-26 K,2.0,2.0,1.0,2.0,2.0,3.0,3.0
4,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],Automotive,Diagnostic/Test engineer,[2018],[2021],E/E Diagnostic Integration Engineer - Automotive,Milan,Engineering,Consultant,Commercial note,NaN,NaN,NaN,"The candidate, inserted within a multidiscipli...",The ideal candidate has a degree in Electronic...,[1-3],26-28K,30-32K,Five-year degree,electronic Engineering,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21372,79993,Hired,26 - 30 years,TORRE ANNUNZIATA » Naples ~ Campania,Male,NaN,X,chemical engineering,Five-year degree,[0],Others,Graduating student,[2023],[2023],Junior Project Engineer (C&Q),Pomezia,Tech Consulting & Solutions,Consultant,HR interview,OK,RS23.0793,~ 3 - High,"The resource, included in a team dedicated to ...",The ideal candidate has a Master's Degree in C...,[0],- 20K,- 20K,Five-year degree,chemical engineering,Pomezia,Not available,Not available,2.0,2.0,3.0,3.0,3.0,3.0,3.0
21373,79993,Hired,26 - 30 years,TORRE ANNUNZIATA » Naples ~ Campania,Male,NaN,X,chemic

## Remove rows

### Drop duplicates

In [84]:
df = df.drop_duplicates().reset_index(drop=True)

### Clean the columns' names

In [85]:
df.columns = [c.strip() for c in df.columns]
df.columns = [c.encode('ascii', 'ignore').decode().strip() for c in df.columns]
df.columns

Index(['ID', 'Candidate State', 'Age Range', 'Residence', 'Sex',
       'Protected category', 'TAG', 'Study area', 'Study Title',
       'Years Experience', 'Sector', 'Last Role', 'Year of insertion',
       'Year of Recruitment', 'Recruitment Request', 'Assumption Headquarters',
       'Job Family Hiring', 'Job Title Hiring', 'event_type__val',
       'event_feedback', 'linked_search__key', 'Overall', 'Job Description',
       'Candidate Profile', 'Years Experience.1', 'Minimum Ral', 'Ral Maximum',
       'Study Level', 'Study Area.1', 'Akkodis headquarters', 'Current Ral',
       'Expected Ral', 'Technical Skills', 'Standing/Position', 'Comunication',
       'Maturity', 'Dynamism', 'Mobility', 'English'],
      dtype='object')

### Create new IDs and separate different people with duplicating IDs

In [86]:
import pandas as pd

unique_ids_before = df['ID'].nunique()

invariant_columns = [
    'ID', 'Sex',
      'Job Title Hiring', 'Study Area.1', 'Assumption Headquarters',
      'Year of insertion', 'Age Range',
    'Study area', 'Study Title', 'Years Experience', 'Residence'
]

# Create a backup of original IDs to avoid unintentional merging
df['Original_ID'] = df['ID']

def split_id(group):
    # Use `drop_duplicates` to check for uniqueness in invariant_columns
    unique_combinations = group[invariant_columns].drop_duplicates()
    
    if len(unique_combinations) == 1:
        # All rows have the same invariant values; no split needed
        group['New_ID'] = group['Original_ID'].iloc[0]
    else:
        # Split into subgroups based on unique combinations of invariant_columns
        for idx, (_, subgroup) in enumerate(group.groupby(invariant_columns, dropna=False)):
            group.loc[subgroup.index, 'New_ID'] = f"{group['Original_ID'].iloc[0]}_{idx + 1}"
    
    return group

# Apply splitting logic
df = df.groupby('Original_ID', group_keys=False).apply(split_id)

# Update ID with New_ID where applicable
df['ID'] = df['New_ID'].fillna(df['Original_ID'])

# Cleanup temporary columns
df = df.drop(columns=['New_ID', 'Original_ID']).reset_index(drop=True)

# Final check
unique_ids_after = df['ID'].nunique()

print(f"🔵 Unique IDs before cleaning: {unique_ids_before}")
print(f"🟢 Unique IDs after cleaning: {unique_ids_after}")
print(f"🧮 Difference: {unique_ids_after - unique_ids_before} new IDs created")


🔵 Unique IDs before cleaning: 12263
🟢 Unique IDs after cleaning: 13372
🧮 Difference: 1109 new IDs created


### Drop redundant columns

In [87]:
df = df.drop(columns=['linked_search__key', 'Year of Recruitment'])


### Remove candidates in first stages

In [88]:
# Normalize columns
df['Candidate State'] = df['Candidate State'].str.strip().str.lower()

# Create a mask for IDs to drop
ids_to_drop = []

grouped = df.groupby('ID')

for id_value, group in grouped:
    is_single_row_group = len(group) == 1
    is_single_row_group=True
    has_imported_or_first_contact = (group['Candidate State'].isin(['imported', 'first contact', 'in selection'])).all()
    
    # Check if all 'Sector' values are NaN
    sector_all_nan = group['Sector'].isna().all()
    
    if has_imported_or_first_contact and sector_all_nan and is_single_row_group:
        ids_to_drop.append(id_value)

# Now filter out the rows with these IDs
df = df[~df['ID'].isin(ids_to_drop)]

# Reset index
df = df.reset_index(drop=True)

print(f"Dropped {len(ids_to_drop)} IDs.")


Dropped 7963 IDs.


### Removal of Candidates with Inconsistent Final Outcomes

In [89]:
state_order = ['imported', 'first contact', 'in selection', 'qm', 'economic proposal', 'vivier', 'hired']
event_order = ['cv request', 'contact note', 'hr interview', 'bm interview', 'technical interview', 
               'qualification meeting', 'economic proposal', 'candidate notification']

# Define a function to sort each group
def sort_group(group):
    # Sort the group by Candidate State and Event_Type__Val based on the specified order
    sorted_group = group.sort_values(by=['Candidate State', 'event_type__val'], 
                                     key=lambda col: col.map(lambda x: (state_order.index(x) if x in state_order else -1, 
                                                                        event_order.index(x) if x in event_order else -1)))
    return sorted_group

# Apply sorting function to each group and update the DataFrame (ordered version)
grouped = df.groupby('ID', group_keys=False).apply(sort_group)

# Reset the index after sorting
df = grouped.reset_index(drop=True)
df

,ID,Candidate State,Age Range,Residence,Sex,Protected category,TAG,Study area,Study Title,Years Experience,Sector,Last Role,Year of insertion,Recruitment Request,Assumption Headquarters,Job Family Hiring,Job Title Hiring,event_type__val,event_feedback,Overall,Job Description,Candidate Profile,Years Experience.1,Minimum Ral,Ral Maximum,Study Level,Study Area.1,Akkodis headquarters,Current Ral,Expected Ral,Technical Skills,Standing/Position,Comunication,Maturity,Dynamism,Mobility,English
0,71470,hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],Automotive,Diagnostic/Test engineer,[2018],E/E Diagnostic Integration Engineer - Automotive,Milan,Engineering,Consultant,Candidate notification,NaN,NaN,"The candidate, inserted within a multidiscipli...",The ideal candidate has a degree in Electronic...,[1-3],26-28K,30-32K,Five-year degree,electronic Engineering,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,71470,hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],Automotive,Diagnostic/Test engineer,[2018],E/E Diagnostic Integration Engineer - Automotive,Milan,Engineering,Consultant,BM interview,NaN,NaN,"The candidate, inserted within a multidiscipli...",The ideal candidate has a degree in Electronic...,[1-3],26-28K,30-32K,Five-year degree,electronic Engineering,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,71470,hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],Automotive,Diagnostic/Test engineer,[2018],E/E Diagnostic Integration Engineer - Automotive,Milan,Engineering,Consultant,Contact note,NaN,NaN,"The candidate, inserted within a multidiscipli...",The ideal candidate has a degree in Electronic...,[1-3],26-28K,30-32K,Five-year degree,electronic Engineering,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,71470,hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],Automotive,Diagnostic/Test engineer,[2018],E/E Diagnostic Integration Engineer - Automotive,Milan,Engineering,Consultant,BM interview,OK,~ 2 - Medium,"The candidate, inserted within a multidiscipli...",The ideal candidate has a degree in Electronic...,[1-3],26-28K,30-32K,Five-year degree,electronic Engineering,Modena,22-24 K,24-26 K,2.0,2.0,1.0,2.0,2.0,3.0,3.0
4,71470,hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],Automotive,Diagnostic/Test engineer,[2018],E/E Diagnostic Integration Engineer - Automotive,Milan,Engineering,Consultant,Commercial note,NaN,NaN,"The candidate, inserted within a multidiscipli...",The ideal candidate has a degree in Electronic...,[1-3],26-28K,30-32K,Five-year degree,electronic Engineering,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11569,62535,in selection,40 - 45 years,CHIETI » Chieti ~ Abruzzo,Female,NaN,"-, ENVIRONMENTAL QUALITY, ENVIRONMENTAL MANAGE...",Engineering for the environment and the territory,Five-year degree,[7-10],Oil and Gas,Project Engineer,[2023],NaN,NaN,NaN,NaN,Candidate notification,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26-28 K,34-36 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11570,79993_1,hired,26 - 30 years,TORRE ANNUNZIATA » Naples ~ Campania,Male,NaN,X,chemical engineering,Five-year degree,[0],Others,Graduating student,[2023],Junior Project Engineer (C&Q),Pomezia,Tech Consulting & Solutions,Consultant,HR interview,OK,~ 3 - High,"The resource, included in a team dedicated to ...",The ideal candidate ha

In [90]:
def remove_not_hired_valid_candidates(df):
    df['Candidate State'] = df['Candidate State'].str.strip().str.lower()
    df['event_type__val'] = df['event_type__val'].str.strip().str.lower()
    df['event_feedback'] = df['event_feedback'].str.strip()
    feedbacks_to_remove = [
        'OK (other candidate)', 
        'KO (lost availability)', 
        'OK (hired)', 
        'OK (waiting for departure)', 
        'KO (opportunity closed)', 
        'KO (retired)', 
        'KO (ral)', 
        'KO (proposed renunciation)'
    ]
    df['Hired'] = df['Candidate State'].apply(
    lambda x: True if x in [
    'hired', 
    # 'economic proposal'
    ] else False)
 
    df = df.groupby('ID', group_keys=False).apply(sort_group)
    df = df.reset_index(drop=True)
    df = df.drop(columns=['Candidate State'])
    last_event = df.groupby('ID').tail(1)
    ids_to_remove_feedback = last_event[(last_event['event_feedback'].isin(feedbacks_to_remove)) & (last_event['Hired'] != True)]['ID'].tolist()

    ids_to_remove_event = last_event[(last_event['event_type__val'].isin(['economic proposal', 'candidate notification'])) & (last_event['Hired'] != True)]['ID'].tolist()

    all_ids_to_remove = set(ids_to_remove_feedback + ids_to_remove_event)

    print(f"Number of unique IDs to remove: {len(all_ids_to_remove)}")

    total_ids_before = df['ID'].nunique()

    df = df[~df['ID'].isin(all_ids_to_remove)].reset_index(drop=True)

    total_ids_after = df['ID'].nunique()

    print(f"Total IDs before cleaning: {total_ids_before}")
    print(f"Total IDs after cleaning: {total_ids_after}")
    print(f"Total IDs removed: {total_ids_before - total_ids_after}")

    df = df.drop(columns=['state_order', 'event_order'], errors='ignore')
    return df

df = remove_not_hired_valid_candidates(df)

Number of unique IDs to remove: 955
Total IDs before cleaning: 5409
Total IDs after cleaning: 4454
Total IDs removed: 955


## Preprocess columns

### Ral Mapping

In [91]:
df.columns

Index(['ID', 'Age Range', 'Residence', 'Sex', 'Protected category', 'TAG',
       'Study area', 'Study Title', 'Years Experience', 'Sector', 'Last Role',
       'Year of insertion', 'Recruitment Request', 'Assumption Headquarters',
       'Job Family Hiring', 'Job Title Hiring', 'event_type__val',
       'event_feedback', 'Overall', 'Job Description', 'Candidate Profile',
       'Years Experience.1', 'Minimum Ral', 'Ral Maximum', 'Study Level',
       'Study Area.1', 'Akkodis headquarters', 'Current Ral', 'Expected Ral',
       'Technical Skills', 'Standing/Position', 'Comunication', 'Maturity',
       'Dynamism', 'Mobility', 'English', 'Hired'],
      dtype='object')

In [92]:
ral_mapping = {
    '- 20 K': 19000,
    '- 20K': 19000,
    '20-22 K': 21000,
    '20-22K': 21000,
    '22-24 K': 23000,
    '22-24K': 23000,
    '24-26 K': 25000,
    '24-26K': 25000,
    '26-28 K': 27000,
    '26-28K': 27000,
    '28-30 K': 29000,
    '28-30K': 29000,
    '30-32 K': 31000,
    '30-32K': 31000,
    '32-34 K': 33000,
    '32-34K': 33000,
    '34-36 K': 35000,
    '34-36K': 35000,
    '36-38 K': 37000,
    '36-38K': 37000,
    '38-40 K': 39000,
    '38-40K': 39000,
    '40-42 K': 41000,
    '40-42K': 41000,
    '42-44 K': 43000,
    '42-44K': 43000,
    '44-46 K': 45000,
    '44-46K': 45000,
    '46-48 K': 47000,
    '46-48K': 47000,
    '48-50 K': 49000,
    '48-50K': 49000,
    '+ 50 K': 55000,
    '+50K': 55000,
    '20K': 20000,
    'Not available': None,
    'Not Avail.': None,
    np.nan: None
}

ral_columns = ['Expected Ral', 'Minimum Ral', 'Ral Maximum', 'Current Ral']

for col in ral_columns:
    if col in df.columns:
        df[col] = df[col].astype(str).map(ral_mapping)
    else:
        print(f"Warning: Column '{col}' not found in the DataFrame.")

### Overall mapping

In [93]:
set(df['Overall'])

{'1 - Low',
 '2 - Medium',
 '3 - High',
 '4 - Top',
 nan,
 '~ 1 - Low',
 '~ 2 - Medium',
 '~ 3 - High',
 '~ 4 - Top'}

In [94]:
score_mapping = {
    '1 - Low': 1,
    '2 - Medium': 2,
    '3 - High': 3,
    '4 - Top': 4,
    '~ 1 - Low': 1,
    '~ 2 - Medium': 2,
    '~ 3 - High': 3,
    '~ 4 - Top': 4
}
df['Overall'] = df['Overall'].map(score_mapping)

### Make the `Protected category` column boolean

In [95]:
df['Protected category'] = df['Protected category'].apply(lambda x: True if 'article' in str(x).lower() else False)

### Remove invalid values from `Job Title Hiring`

In [96]:
df['Job Title Hiring'] = df['Job Title Hiring'].replace('???', None)

In [97]:
sum(df['Job Title Hiring'].isna()), len(df['Job Title Hiring']), sum(df['Job Title Hiring']==None)

(6016, 8039, 0)

### Residence Column

In [98]:
df[df['Residence'].isna()]['ID'].nunique()

0

In [99]:
df['Residence'].fillna('ITALY » (STATE) ~ Italy', inplace=True)

In [100]:
city_mapping = {'Syracuse':'Siracusa', 'Padua':'Padova',
                'Power':'Potenza',
                "Sant'agata di Militello": "Sant'Agata di Militello",
                "Reggio in Emilia": "Reggio nell'Emilia",
                "Palazzo San Gevasio": "Palazzo San Gervasio",
                "San Nicolao Valley": "Valle San Nicolao",
                "Sweaters": "Maglie",  
                "Vast": "Vasto",
                "Citta' Sant'angelo": "Città Sant'Angelo",
                "San Vito Dei Normanni": "San Vito dei Normanni",
                "Reggio di Calabria": "Reggio Calabria",
                "Acquaviva Delle Fonti": "Acquaviva delle Fonti",
                "Boxes in Pittari": "Caselle in Pittari",
                "Santo Stino di Livenza": "San Stino Livenza",
                "Seas": "Marene",
                "Sant'antonio Abate":"Sant'Antonio Abate","Lions":"Lioni",
                "Monopolies":"Monopoli", "Urago D'oglio":"Urago d'Oglio",
                "San Stino Livenza": "San Stino di Livenza",
                "Alpinolo Hospital": "Mercogliano",
                "Lower Venue": "Luino",
                "Scroll": "Scorrano",
                "Walnuts": "Noicattaro",
                "The Loggia": "La Loggia",
                "Yield": "Rende",
                "Cava De' Tirreni": "Cava de' Tirreni",
                "Trino Vercellese": "Trino",
                "San Felice Sul Panaro": "San Felice sul Panaro",
                "San Paolo Beautiful Site": "San Paolo Bel Sito",
                    "Montecatini Terme": "Montecatini Terme",
    "San Marco D'alunzio": "San Marco d'Alunzio",
    "Hospitaletto": "Hospitaletto",
    "Casalnuovo di Naples": "Casalnuovo di Napoli",
    "Pomigliano D'arco": "Pomigliano d'Arco",
    "Touch Of Casauria": "Tocco da Casauria",
    "Nardo'": "Nardò",
    "Ligurian Stone": "Pietra Ligure",
    "Dinner Below": "Cene",
    "Bornate": "Bornate",
    "Chors": "Cori",
    "Seek": "Cerda",    "Cava de' Tirreni": "Cava de' Tirreni",
    "Rivalta di Turin": "Rivalta di Torino",
    "Rounds": "Rotondi",
    "Melito di Naples": "Melito di Napoli",
    "Sant'alessio Siculo": "Sant'Alessio Siculo",
    "Canonica D'adda": "Canonica d'Adda",
    "Zane'": "Zanè",
    "Island Of Female": "Isola delle Femmine",
    "Casalnuovo di Naples": "Casalnuovo di Napoli",
    "Arzago D'adda": "Arzago d'Adda",
    "Oil Bridge": "Ponte dell'Olio",
    "Aci Sant'antonio": "Aci Sant'Antonio",
       "PO CUT": "Taglio di Porto Viro",
    "L'AQUILA": "L'Aquila",
    "VENETIAN EEL": "Padova",
    "CAVA DE' TIRRENI": "Cava de' Tirreni",
    "RIVANAZZANO TERME": "Rivanazzano Terme",
    "FARMHOUSE": "Pisa",
    "FOURTH": "Napoli",
    "BORNATE": "Bornate",
    "CATHOLIC": "Cattolica",
    "RONCHI DEI LEGIONARIE": "Ronchi dei Legionari",
    "CREAM": "Crema",
    "BORGO D'ALE": "Borgo d'Ale",
    "AFFORI": "Milano",
    "SAN STINO LIVENZA": "San Stino di Livenza",
    "MIRROR": "Specchia",
    "BARCELONA POZZO DI GOTTO": "Barcellona Pozzo di Gotto",
    "MARINE": "Roma",
    "SLOW ON THE SEVESO": "Seveso",
    "SAN GENESIO ED UNITI": "San Genesio ed Uniti",
    "FORTRESS": "Fortezza",
    "LUPIA CAMPAIGN": "Campagna Lupia",
    "VALENTINE'S DAY IN ABRUZZO CITERIORE": "Pescara",}
def city_transform(city):
    city = ' '.join([c.capitalize() if c.upper() not in ['DI','IN','DEL','A'] else c.lower() for c in city.split()])
    if city.strip() in city_mapping:
        city = city_mapping[city.strip()]
    return city

def parse_residence(residence):
    try:
        parts = residence.split('»')
    except:
        return pd.Series([None, None, None, None, False])
    city = parts[0].strip()
    if len(parts) < 2 or '~' not in parts[1]:
        italian_residence = (city.upper() == 'ITALY')  
 
        return pd.Series([None, None, None, city_transform(city), italian_residence])
    subparts = parts[1].split('~')
    province = subparts[0].strip()
    region = subparts[1].strip() if len(subparts) > 1 else None
    if province == '(COUNTRY)' or province == '(STATE)':
        country = city
    else:
        country = 'ITALY'

    country = country.capitalize()
    region = region.capitalize()
    province = province.capitalize()
    city = city_transform(city)

    if country.upper() == 'ITALY':
        italian_residence = True
    else:
        italian_residence = False
        region = None
        province = None
        city = None
    
    return pd.Series([country, region, province, city, italian_residence])

df[['Residence Country', 'Residence Italian Region', 'Residence Italian Province', 'Residence Italian City', 'Italian Residence']] = df['Residence'].apply(parse_residence)

# # Binary flags
# df['Italian Residence'] = df['Residence Country'].apply(lambda x: 1 if x.lower() == 'italy' else 0)
# df['European Residence'] = df['Residence Country'].apply(lambda x: 1 if x.lower() in ['italy', 'france', 'germany'] else 0)
european_countries = {
    'ALBANIA', 'AUSTRIA', 'BELARUS', 'BELGIUM', 'BULGARIA', 'CROATIA', 'CZECH REPUBLIC',
    'FRANCE', 'GERMANY', 'GREECE', 'LITHUANIA', 'MALTA', 'MONACO', 'NETHERLANDS',
    'PORTUGAL', 'REPUBLIC OF POLAND', 'ROMANIA', 'RUSSIAN FEDERATION', 'SAN MARINO',
    'SERBIA AND MONTENEGRO', 'SLOVAKIA', 'SPAIN', 'SWEDEN', 'SWITZERLAND', 'UKRAINE',
    'GREAT BRITAIN-NORTHERN IRELAND', 'YUGOSLAVIA', 'ITALY','TÜRKIYE', 'USSR'
}
df['European Residence'] = df['Residence Country'].apply(lambda x: x in european_countries if pd.notna(x) else False)


In [101]:
df[df['Akkodis headquarters']!='not specified']['Akkodis headquarters']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
8034    NaN
8035    NaN
8036    NaN
8037    NaN
8038    NaN
Name: Akkodis headquarters, Length: 8039, dtype: object

In [102]:
pd.read_csv('countries.csv')

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia


In [103]:
df['Residence Italian City']

0       Perch Dosimo
1               None
2            Bologna
3           Avezzano
4             Naples
            ...     
8034            Rome
8035           Turin
8036           Turin
8037           Turin
8038           Milan
Name: Residence Italian City, Length: 8039, dtype: object

In [104]:
import json
import urllib
import requests

where = urllib.parse.quote_plus("""
{
    "name": "Sant'Antonio Abate"
}
""")
url = 'https://parseapi.back4app.com/classes/City?limit=10&where=%s' % where
headers = {
    'X-Parse-Application-Id': 'rPfDpoNwAXlUjYrLAYtkVa6HXYcorAOJ9pefs00V', # This is the fake app's application id
    'X-Parse-Master-Key': 'rpXD45YgCcmIyLf13fwUsguY9hRPaiH4xaIPsQLT' # This is the fake app's readonly master key
}
data = json.loads(requests.get(url, headers=headers).content.decode('utf-8')) # Here you have the data that you need

data['results']

[{'objectId': 'VaNtPFh2cC',
  'location': {'__type': 'GeoPoint',
   'latitude': 40.72152,
   'longitude': 14.54021},
  'cityId': 3167393,
  'name': "Sant'Antonio Abate",
  'country': 'Italy',
  'countryCode': 'IT',
  'muni': '063074',
  'featureCode': 'PPLA3',
  'adminCode': '04',
  'population': 18859,
  'createdAt': '2019-12-13T03:16:48.483Z',
  'updatedAt': '2019-12-13T03:16:48.483Z'}]

### Aggregate Records

In [105]:
import pandas as pd
import re

def clean_text(text):
    if not isinstance(text, str) or not text.strip():
        return None
    # Remove leading dashes, bullets, and whitespace
    if text.startswith('o '):
        text = text[1:].strip()
    text = re.sub(r'^[\-\•\*]+\s*', '', text.strip())
    # Normalize multiple spaces to a single space
    text = re.sub(r'\s+', ' ', text)
    # Optionally, lowercase everything
    text = text.lower().strip()
    return text
    # from autocorrect import Speller

    # spell = Speller()
    # return spell(text)
    

for col in ['Candidate Profile','Last Role','Job Description','Candidate Profile']:
    df[col] = df[col].apply(clean_text)

In [106]:
def find_differences_by_id(df):
    ignore_columns = {'Job Description','event_feedback', 'event_type__val', "Overall", "Minimum Ral",'Ral Maximum', "Technical Skills", "Mobility", "English","Dynamism","Maturity","Comunication","Standing/Position"}
    id_groups = df.groupby('ID')
    
    for id_val, group in id_groups:
        if len(group) <= 1:
            continue

        differing_cols = []
        for col in df.columns:
            if col in ignore_columns or col == 'ID':
                continue
            unique_vals = group[col].dropna().unique()
            if len(unique_vals) > 1:
                differing_cols.append((col, unique_vals))

        if differing_cols:
            print(f"\nID: {id_val}")
            # if len(differing_cols)>1:
                # print(group)
            for col, vals in differing_cols:
                print(f"  → Column '{col}' differs: {list(vals)}")
            for v in vals:
                print(f'\t{v}')
            # print(group)

find_differences_by_id(df)



ID: 14915
  → Column 'Candidate Profile' differs: ['excellent references for product/subsystem v&v activities for the railway sector excellent references in conducting product/system validation/integration test activities good knowledge of the alstom v&v process good knowledge of the functionality of tms systems knowledge is appreciated of products good familiarity with laboratory testing activities (in particular for functional tests on targets)', 'excellent references for product/subsystem v&v activities for the railway sector excellent references in conducting product/system validation/integration test activities good knowledge of the alstom v&v process good knowledge of the functionality of tms systems knowledge is appreciated of alstom products good familiarity with laboratory testing activities (in particular for functional tests on targets)']
	excellent references for product/subsystem v&v activities for the railway sector excellent references in conducting product/system valid

In [107]:
import numpy as np
def aggregate_group(group):
    for col in group.columns:
        if col == 'ID':
            continue  
        values = group[col].dropna().unique()
        if len(values) == 0:
            continue 
        elif len(values) == 1:
            group[col] = values[0]
        else:
            if np.issubdtype(group[col].dropna().dtype, np.number):
                avg_value = group[col].dropna().astype(float).mean()
                group[col] = avg_value
            else:
                string_values = [str(v).strip() for v in values]
                filtered_values = [v for v in string_values if v]
                combined_string = "|".join(str(v) for v in filtered_values)
                group[col] = combined_string
    return group

def aggregate_all_records(df):
    df_cleaned = df.drop(columns=['event_feedback', 'event_type__val']).drop_duplicates().reset_index(drop=True)

    grouped = df_cleaned.groupby('ID')

    groups_with_multiple = grouped.filter(lambda x: len(x) > 1)

    fixed_groups = groups_with_multiple.groupby('ID', group_keys=False).apply(aggregate_group).drop_duplicates().reset_index(drop=True)
    groups_with_single = grouped.filter(lambda x: len(x) == 1)
    final_df = pd.concat([fixed_groups, groups_with_single], ignore_index=True)

    print(f"Original number of records: {len(df['ID'])}")
    print(f"Aggregated number of records: {len(final_df['ID'])}")
    return final_df


final_df = aggregate_all_records(df)
# print_pipe_entries(new_df)

Original number of records: 8039
Aggregated number of records: 4454


In [108]:
def clean_piped_string(val):
    if isinstance(val, str) and '|' in val:
        parts = val.split('|')
        filtered_parts = [p.strip() for p in parts if p.strip()]

        if not filtered_parts:
            return '' 
        elif len(filtered_parts) == 1:
            return filtered_parts[0]
        else:
            return '|'.join(filtered_parts)
    else:
        return val

def clean_all_piped_entries(df):

    df_cleaned = df.copy() 
    string_cols = df_cleaned.select_dtypes(include=['object', 'string']).columns

    print(f"Applying cleaning to columns: {list(string_cols)}")

    for col in string_cols:
        df_cleaned[col] = df_cleaned[col].map(clean_piped_string, na_action='ignore')

    print("Cleaning complete.")
    return df_cleaned
final_df = clean_all_piped_entries(final_df)

Applying cleaning to columns: ['ID', 'Age Range', 'Residence', 'Sex', 'TAG', 'Study area', 'Study Title', 'Years Experience', 'Sector', 'Last Role', 'Year of insertion', 'Recruitment Request', 'Assumption Headquarters', 'Job Family Hiring', 'Job Title Hiring', 'Job Description', 'Candidate Profile', 'Years Experience.1', 'Study Level', 'Study Area.1', 'Akkodis headquarters', 'Residence Country', 'Residence Italian Region', 'Residence Italian Province', 'Residence Italian City']
Cleaning complete.


In [109]:
def print_pipe_entries(df):
    for idx, row in df.iterrows():
        for col in df.columns:
            val = row[col]
            if isinstance(val, str) and '|' in val:
                print(f"Column: {col}, Row ID: {idx}, Value: {val}")
                print(val.split('|'))
                print([v.strip() for v in val.split('|') if v.strip()])
print_pipe_entries(final_df)

Column: Candidate Profile, Row ID: 221, Value: excellent references for product/subsystem v&v activities for the railway sector excellent references in conducting product/system validation/integration test activities good knowledge of the alstom v&v process good knowledge of the functionality of tms systems knowledge is appreciated of products good familiarity with laboratory testing activities (in particular for functional tests on targets)|excellent references for product/subsystem v&v activities for the railway sector excellent references in conducting product/system validation/integration test activities good knowledge of the alstom v&v process good knowledge of the functionality of tms systems knowledge is appreciated of alstom products good familiarity with laboratory testing activities (in particular for functional tests on targets)
['excellent references for product/subsystem v&v activities for the railway sector excellent references in conducting product/system validation/inte

In [110]:
final_df.columns

Index(['ID', 'Age Range', 'Residence', 'Sex', 'Protected category', 'TAG',
       'Study area', 'Study Title', 'Years Experience', 'Sector', 'Last Role',
       'Year of insertion', 'Recruitment Request', 'Assumption Headquarters',
       'Job Family Hiring', 'Job Title Hiring', 'Overall', 'Job Description',
       'Candidate Profile', 'Years Experience.1', 'Minimum Ral', 'Ral Maximum',
       'Study Level', 'Study Area.1', 'Akkodis headquarters', 'Current Ral',
       'Expected Ral', 'Technical Skills', 'Standing/Position', 'Comunication',
       'Maturity', 'Dynamism', 'Mobility', 'English', 'Hired',
       'Residence Country', 'Residence Italian Region',
       'Residence Italian Province', 'Residence Italian City',
       'Italian Residence', 'European Residence'],
      dtype='object')

### Create final dataset

In [114]:
def extract_candidate_features(df):
    columns_to_keep = [
        'Age Range','Years Experience', 'Years Experience.1', 'Sex', 'Protected category', 'Minimum Ral', 'Ral Maximum',
        'Current Ral', 'Expected Ral', 'Technical Skills', 'Standing/Position',
        'Comunication', 'Maturity', 'Dynamism', 'Mobility', 'English', 'Hired',
    ]
    columns_to_keep = df.columns
    print("Extra columns in df:", set(df.columns) - set(columns_to_keep)) 

    df_copy = df.copy()

    cat_order = {
        'Age Range': ['< 20 years', '20 - 25 years', '26 - 30 years', '31 - 35 years', '36 - 40 years', '40 - 45 years', '> 45 years'],
        'Years Experience': ['[0]', '[0-1]', '[1-3]', '[3-5]', '[5-7]', '[7-10]', '[+10]'],
        'Years Experience.1': ['[0]', '[0-1]',  '[1-3]', '[3-5]', '[5-7]', '[7-10]','[+10]'],
    }

    for col, order in cat_order.items():
        if col in df_copy.columns:
            df_copy[col] = pd.Categorical(df_copy[col], categories=order, ordered=True)
            # Convert to numeric codes starting from 1 (instead of 0); missing values become NaN
            df_copy[col] = df_copy[col].cat.codes.replace(-1, pd.NA) + 1
    
    df_copy['Years Experience.1'] = df_copy['Years Experience.1'].fillna(4)
    existing_columns = [col for col in columns_to_keep if col in df_copy.columns]
    return df_copy[existing_columns]


dataset = extract_candidate_features(final_df)
dataset

Extra columns in df: set()


,ID,Age Range,Residence,Sex,Protected category,TAG,Study area,Study Title,Years Experience,Sector,Last Role,Year of insertion,Recruitment Request,Assumption Headquarters,Job Family Hiring,Job Title Hiring,Overall,Job Description,Candidate Profile,Years Experience.1,Minimum Ral,Ral Maximum,Study Level,Study Area.1,Akkodis headquarters,Current Ral,Expected Ral,Technical Skills,Standing/Position,Comunication,Maturity,Dynamism,Mobility,English,Hired,Residence Country,Residence Italian Region,Residence Italian Province,Residence Italian City,Italian Residence,European Residence
0,137,1,NAPLES » Naples ~ Campania,Female,False,PROJECT MANAGEMENT,Management Engineering,Five-year degree,1,Others,project manager,[2023],NaN,NaN,NaN,NaN,2.000000,None,None,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,2.000000,2.000000,2.000000,3.0,3.000000,3.0,False,Italy,Campania,Naples,Naples,True,False
1,165,3,SESTO FIORENTINO » Florence ~ Tuscany,Male,False,"ANGULAR, JAVASCRIPT.",Informatics,Three-year degree,3,Telecom,front end developer,[2023],Front End Developer,Bologna,Tech Consulting & Solutions,Consultant,3.000000,web developer – 03/06/23 • have 2+ years of ex...,web developer – 03/06/23 • have 2+ years of ex...,3,23000.0,29000.0,Three-year degree,Informatics,Milan,23000.0,27000.0,2.500000,3.000000,3.000000,3.000000,3.0,1.000000,3.0,True,Italy,Tuscany,Florence,Sesto Fiorentino,True,False
2,243,4,CARMAGNOLA » Turin ~ Piedmont,Male,False,NaN,Information Engineering,Five-year degree,4,Services and Information Systems,fullstack developer,[2022],NaN,NaN,NaN,NaN,2.666667,None,None,4,NaN,NaN,NaN,NaN,NaN,41000.0,45000.0,2.333333,2.333333,2.333333,1.666667,2.0,2.000000,3.0,False,Italy,Piedmont,Turin,Carmagnola,True,False
3,307,4,LEGNANO » Milan ~ Lombardy,Male,False,NaN,chemical engineering,Five-year degree,6,Oil and Gas,process leader,[2021],NaN,NaN,NaN,NaN,3.000000,None,None,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,2.666667,3.000000,3.000000,3.0,2.666667,3.0,False,Italy,Lombardy,Milan,Legnano,True,False
4,315,1,TURIN » Turin ~ Piedmont,Male,False,"JAVA, ORACLE, EXCEL, OFFICE, EDGE, VISUAL",Management Engineering,Five-year degree,1,Consulting,student,[2023],NaN,NaN,NaN,NaN,2.000000,None,None,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.500000,2.000000,2.500000,2.000000,2.0,3.000000,2.5,False,Italy,Piedmont,Turin,Turin,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4449,9139_2,7,CHIETI » Chieti ~ Abruzzo,Male,False,NaN,Informatics,Five-year degree,7,Others,software developer,[2021],NaN,NaN,NaN,NaN,NaN,None,None,4,NaN,NaN,NaN,NaN,NaN,45000.0,49000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Italy,Abruzzo,Chieti,Chieti,True,False
4450,946_2,3,TREVISO » Treviso ~ Veneto,Male,False,NaN,Informatics,Five-year degree,1,Others,recent graduate/internship during studies,[2023],NaN,NaN,NaN,NaN,NaN,None,None,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Italy,Veneto,Treviso,Treviso,True,False
4451,9761_2,1,TURIN » Turin ~ Piedmont,Male,False,NaN,Automation/Mechatronics Engineering,Five-year degree,7,Others,.,[2022],NaN,NaN,NaN,NaN,NaN,None,None,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Italy,Piedmont,Turin,Turin,True,False
4452,9912_2,3,ROME » Rome ~ Lazio,Male,False,PYTHON,computer engineering,Five-year degree,1,Others,fresh graduate,[2023],NaN,NaN,NaN,NaN,NaN,None,None,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Italy,Lazio,Rome,Rome,True,False


In [75]:
# for i in final_df.columns:
#     print(f'{i} {final_df[i].isna().sum()}, {final_df[i].isna().sum()/len(final_df)}, {set(final_df[i])}')

In [115]:
import pandas as pd
import numpy as np

def fill_missing_values(df):
    df_filled = df.copy()

    for col in df_filled.columns:
        col_dtype = df_filled[col].dtype

        if pd.api.types.is_bool_dtype(col_dtype):
            df_filled[col] = df_filled[col].fillna(False)

        elif pd.api.types.is_numeric_dtype(col_dtype):
            # If it's numeric (int or float), fill with mean
            mean_val = df_filled[col].mean()
            df_filled[col] = df_filled[col].fillna(mean_val)

        elif pd.api.types.is_string_dtype(col_dtype) or df_filled[col].dtype == object:
            df_filled[col] = df_filled[col].fillna('')

        else:
            # Default fallback
            df_filled[col] = df_filled[col].fillna(np.nan)

    return df_filled

# Use the function
final_df_filled = fill_missing_values(dataset)


In [117]:
set(final_df_filled["Years Experience.1"])

{1, 2, 3, 4, 5, 6, 7}

In [119]:


# df_features = final_df.copy()
df_features = final_df_filled.copy()

# df_features = parse_and_convert_numeric_cols(df_features)

df_features['experience_match_score'] = calculate_experience_match_score(df_features)

df_features['salary_fit_score'] = calculate_salary_fit_score(df_features)

df_features = prepare_nlp_text_columns(df_features)

# 5. Calculate NLP Similarity (Choose one or both methods)

# Method A: Using Sentence Transformers
# Load the ST model once before calling the function
try:
    print("\n--- Loading Sentence Transformer Model ---")
    st_model = SentenceTransformer('all-MiniLM-L6-v2')
    print("--- Model Loaded ---")
    df_features['profile_nlp_similarity_st'] = calculate_nlp_similarity_st(df_features, st_model)
    print("--- ST NLP Similarity Calculation Finished ---")
except Exception as e:
    print(f"Error loading/using ST model: {e}. ST NLP similarity not calculated.")
    df_features['profile_nlp_similarity_st'] = np.nan
    st_model = None # Ensure model is None if loading failed


# Method B: Using TF-IDF
# Fit TF-IDF Vectorizer on all texts once before calling the function
try:
    print("\n--- Fitting TF-IDF Vectorizer ---")
    all_texts_for_tfidf = df_features['candidate_text'].tolist() + df_features['job_text'].tolist()
    # You can customize TfidfVectorizer parameters here (e.g., stop_words, max_features)
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
    tfidf_vectorizer.fit(all_texts_for_tfidf)
    print("--- Vectorizer Fitted ---")

    df_features['profile_nlp_similarity_tfidf'] = calculate_nlp_similarity_tfidf(df_features, tfidf_vectorizer)
    print("--- TF-IDF NLP Similarity Calculation Finished ---")
except Exception as e:
     print(f"Error fitting/using TF-IDF: {e}. TF-IDF NLP similarity not calculated.")
     tfidf_vectorizer = None # Ensure vectorizer is None if fitting failed
     df_features['profile_nlp_similarity_tfidf'] = np.nan


df_features = df_features.drop(columns=['candidate_text', 'job_text'])



distance_series, proximity_series = calculate_geo_features(df_features)
df_features['residence_hq_distance_km'] = distance_series
df_features['residence_hq_proximity_score'] = proximity_series

nlp_score_col_for_overall = None
if 'profile_nlp_similarity_st' in df_features.columns and not df_features['profile_nlp_similarity_st'].isnull().all():
    nlp_score_col_for_overall = 'profile_nlp_similarity_st'
elif 'profile_nlp_similarity_tfidf' in df_features.columns and not df_features['profile_nlp_similarity_tfidf'].isnull().all():
     nlp_score_col_for_overall = 'profile_nlp_similarity_tfidf'

score_columns_for_overall = [
    'experience_match_score',
    'salary_fit_score',
    'residence_hq_proximity_score',
    'Overall_scaled' # Note: 'Overall_scaled' is created within calculate_overall_score if 'Overall' exists
]
if nlp_score_col_for_overall:
    score_columns_for_overall.append(nlp_score_col_for_overall)

df_features['overall_match_score'] = calculate_overall_score(df_features, score_columns_for_overall)


df_features['Hired'] = (df_features['Hired'] == True)
print("\n'Hired' column cleaned and added.")


# --- 8. Drop Source/Intermediate Columns ---
# Define the list of final columns you want to keep
columns_to_keep_final = [
    '# TAG', # Identifier
    # 'Italian Residence', 'European Residence', # Booleans
    # 'Overall', 
    'Technical Skills', 'Standing/Position', # Original Scores
    'Comunication', 'Maturity', 'Dynamism', 'Mobility', 'English', # Original Scores
    'Current Ral', 'Expected Ral', 'Minimum Ral', 'Ral Maximum', # Original RALs (now numeric/NaN)
    'Years Experience_parsed', 'Years Experience.1_parsed', # Parsed Years Experience
    'experience_match_score', 'salary_fit_score', # New Scores
    # 'profile_nlp_similarity_st', 
    # 'profile_nlp_similarity_tfidf', # Both NLP Scores (keep both or choose one)
    # 'residence_hq_distance_km', 'residence_hq_proximity_score', # New Geo Scores
    # 'overall_match_score', # New Combined Score
    'Hired' # Target variable
]

df_features_cleaned = drop_columns_except(df_features, columns_to_keep_final)

feature_columns_for_ml = [col for col in df_features_cleaned.columns if col != 'Hired']

X = df_features_cleaned[feature_columns_for_ml]
y = df_features_cleaned['Hired']
for c in X.columns:
    print(c,X[c].isna().sum())
# from imblearn.over_sampling import SMOTE
# print(f'Before resampling:{len(X)}')

# smote = SMOTE(random_state=42)
# X, y = smote.fit_resample(X, y)
# print(f'After resampling:{len(X)}')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y) # Stratify to maintain hire ratio
# index_cut=int(0.65 * len(X))
# X_train, X_test = X[:index_cut], X[index_cut:]
# y_train, y_test = y[:index_cut], y[index_cut:]

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# model = HistGradientBoostingClassifier(random_state=42)

# Train the model - pass X_train directly (it may contain NaNs)
print("\n--- Training HistGradientBoostingClassifier Model ---")
model.fit(X_train, y_train)
print("--- Model Training Complete ---")


# --- Make Predictions and Evaluate ---
# Predict - pass X_test directly (it may contain NaNs)
y_pred = model.predict(X_test)

print("\n--- Model Evaluation ---")
print("Accuracy:", accuracy_score(y_test, y_pred))

# Calculate F1 Score specifically
f1 = f1_score(y_test, y_pred)
print("F1 Score (for True/Hired class):", f1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# --- Final Output ---
print("\n--- Processed DataFrame (first 5 rows) ---")
print(df_features_cleaned.head())
print("\n--- Info on Processed DataFrame ---")
df_features_cleaned.info()
print("\n--- Columns in Processed DataFrame ---")
print(df_features_cleaned.columns.tolist())


--- Loading Sentence Transformer Model ---
--- Model Loaded ---
--- ST NLP Similarity Calculation Finished ---

--- Fitting TF-IDF Vectorizer ---
--- Vectorizer Fitted ---
--- TF-IDF NLP Similarity Calculation Finished ---
Attempting to geocode HQ location: 'Via dei Fornari 12, Bologna, Italy'
Error: Could not geocode HQ location 'Via dei Fornari 12, Bologna, Italy'. Cannot calculate geographical features.

'Hired' column cleaned and added.
Minimum Ral 0
Ral Maximum 0
Current Ral 0
Expected Ral 0
Technical Skills 0
Standing/Position 0
Comunication 0
Maturity 0
Dynamism 0
Mobility 0
English 0
salary_fit_score 0

--- Training HistGradientBoostingClassifier Model ---
--- Model Training Complete ---

--- Model Evaluation ---
Accuracy: 0.8857142857142857
F1 Score (for True/Hired class): 0.14285714285714285

Classification Report:
              precision    recall  f1-score   support

       False       0.89      0.99      0.94      1300
        True       0.54      0.08      0.14       170

In [ ]:
for x in final_df_filled.columns:
    print(x,final_df_filled[x].isna().sum())

ID 0
Age Range 0
Residence 0
Sex 0
Protected category 0
TAG 0
Study area 0
Study Title 0
Years Experience 0
Sector 0
Last Role 0
Year of insertion 0
Recruitment Request 0
Assumption Headquarters 0
Job Family Hiring 0
Job Title Hiring 0
Overall 0
Job Description 0
Candidate Profile 0
Years Experience.1 0
Minimum Ral 0
Ral Maximum 0
Study Level 0
Study Area.1 0
Akkodis headquarters 0
Current Ral 0
Expected Ral 0
Technical Skills 0
Standing/Position 0
Comunication 0
Maturity 0
Dynamism 0
Mobility 0
English 0
Hired 0


In [208]:
import pandas as pd

# Combine X_train and y_train for correlation
df_corr = X_train.copy()
df_corr['Hired'] = y_train.astype(int)  # convert boolean to 0/1 for correlation

# Compute correlation between each feature and the target
correlations = df_corr.corr(numeric_only=True)['Hired'].drop('Hired').sort_values(ascending=False)

print("\n--- Correlation with 'Hired' ---")
print(correlations)



--- Correlation with 'Hired' ---
Dynamism                        0.211579
Maturity                        0.189731
Comunication                    0.177565
Standing/Position               0.156333
Technical Skills                0.152875
English                         0.123750
Mobility                        0.098411
Years Experience_parsed        -0.004796
Current Ral                    -0.114914
Expected Ral                   -0.227532
Minimum Ral                          NaN
Ral Maximum                          NaN
Years Experience.1_parsed            NaN
experience_match_score               NaN
salary_fit_score                     NaN
residence_hq_distance_km             NaN
residence_hq_proximity_score         NaN
Name: Hired, dtype: float64


In [209]:
(y_test==True).sum(),len(y_test)

(np.int64(14), 1337)

In [210]:
(y_train==True).sum(),len(y_train)

(np.int64(500), 3117)

In [191]:
X_train

,Overall,Minimum Ral,Ral Maximum,Current Ral,Expected Ral,Technical Skills,Standing/Position,Comunication,Maturity,Dynamism,Mobility,English,Years Experience_parsed,Years Experience.1_parsed,experience_match_score,salary_fit_score,profile_nlp_similarity_st,profile_nlp_similarity_tfidf,residence_hq_distance_km,residence_hq_proximity_score,overall_match_score
2991,2.000000,NaN,NaN,29000.0,31000.0,3.000000,2.000000,3.000000,2.000000,2.000000,1.0,3.0,10.0,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.200000
953,1.000000,NaN,NaN,NaN,NaN,1.000000,2.000000,2.000000,2.000000,1.000000,1.0,2.0,0.0,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.100000
1404,3.000000,NaN,NaN,NaN,NaN,2.000000,2.333333,2.666667,2.666667,2.333333,2.0,3.0,0.0,0.0,1.0,NaN,0.345870,0.018301,NaN,NaN,0.648623
3155,NaN,NaN,NaN,27000.0,31000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000
3219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,3.333333,NaN,NaN,45000.0,47000.0,3.333333,2.666667,2.666667,3.333333,2.666667,1.0,2.0,10.0,NaN,NaN,NaN,0.414359,0.000000,NaN,NaN,0.540513
2557,NaN,NaN,NaN,37000.0,45000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000
2143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000
77,2.000000,NaN,NaN,NaN,NaN,2.000000,2.000000,2.000000,2.000000,2.000000,2.0,2.0,2.0,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.200000


In [71]:


# --- Create a copy of the DataFrame for processing ---
df_features = final_df.copy()

def parse_experience_string(exp_str):
    if pd.isna(exp_str):
        return np.nan
    if not isinstance(exp_str, str):
         # If it's already a number (e.g., int/float not converted to str yet), return it
         return float(exp_str)

    exp_str = exp_str.strip()
    if not exp_str:
        return np.nan

    # Handle combined formats like '[3-5] | [1-3]'
    parts = exp_str.split('|')
    values = []

    for part in parts:
        part = part.strip().replace('[', '').replace(']', '') # Remove brackets
        if not part: continue

        try:
            if '-' in part:
                if part.startswith('+'): # Handle '+10' case explicitly if it appears as '+10'
                     value = float(part.replace('+', '')) # Treat +10 as 10
                else:
                    low, high = map(float, part.split('-'))
                    value = (low + high) / 2.0 # Midpoint of range
            elif part.startswith('+'): # Handle '+10' when it's just '+10' after stripping []
                 value = float(part.replace('+', '')) # Treat +10 as 10
            else:
                value = float(part) # Handle single numbers like '0', '5', '10'

            values.append(value)
        except ValueError:
            # print(f"Warning: Could not parse experience string part '{part}' from '{exp_str}'")
            continue # Ignore parts that cannot be parsed

    if values:
        # For multiple ranges/values separated by '|', take the maximum experience
        return max(values)
    else:
        return np.nan # Return NaN if no parts could be parsed

# Apply custom parsing to experience columns
df_features['Years Experience_parsed'] = df_features['Years Experience'].apply(parse_experience_string)
df_features['Years Experience.1_parsed'] = df_features['Years Experience.1'].apply(parse_experience_string)

# Convert RAL columns and Overall to numeric, coercing errors
ral_overall_cols = ['Current Ral', 'Expected Ral', 'Minimum Ral', 'Ral Maximum', 'Overall']
for col in ral_overall_cols:
     # Simple conversion. If 'k' format exists, this will turn it to NaN.
     # A more complex parser would be needed for 'k'. Sticking to simple numeric/NaN for now.
     df_features[col] = pd.to_numeric(df_features[col], errors='coerce')


print("\n--- DataFrame after parsing and converting numeric columns (first 5 rows) ---")
print(df_features[['Years Experience', 'Years Experience_parsed', 'Years Experience.1', 'Years Experience.1_parsed', 'Current Ral', 'Expected Ral', 'Minimum Ral', 'Ral Maximum']].head())
print("\n--- DataFrame Info after parsing and converting numeric columns ---")
df_features[['Years Experience_parsed', 'Years Experience.1_parsed'] + ral_overall_cols].info()


# --- 2. Feature: Experience Match Score ---
# Simple inverse of difference, scaled. Smaller difference = higher score.
# Now use the parsed numeric columns
def calculate_experience_match(candidate_exp_parsed, job_req_exp_parsed):
    # pd.isna handles both np.nan and None
    if pd.isna(candidate_exp_parsed) or pd.isna(job_req_exp_parsed):
        return np.nan
    diff = abs(candidate_exp_parsed - job_req_exp_parsed)
    # Score decreases as difference increases, never 0 unless diff is infinite
    return 1 / (diff + 1)

df_features['experience_match_score'] = df_features.apply(
    lambda row: calculate_experience_match(row['Years Experience_parsed'], row['Years Experience.1_parsed']),
    axis=1
)


# --- 3. Feature: Salary Fit Score ---
# Score based on whether expected RAL is within the min/max range.
# Using the converted RAL columns
def calculate_salary_fit(expected_ral, min_ral, max_ral):
    # pd.isna handles both np.nan and None
    if pd.isna(expected_ral) or pd.isna(min_ral) or pd.isna(max_ral):
        return np.nan

    if min_ral > max_ral: # Handle illogical ranges
         return np.nan

    if expected_ral >= min_ral and expected_ral <= max_ral:
        return 1.0 # Perfect fit

    # If outside the range, score decreases with distance from the range
    if expected_ral < min_ral:
        distance = min_ral - expected_ral
    else: # expected_ral > max_ral
        distance = expected_ral - max_ral

    # Scale the distance (adjust denominator based on expected RAL scale)
    # Using the range size + min_ral as a scaling factor
    range_size = max_ral - min_ral
    scale_factor = range_size if range_size > 0 else min_ral # Avoid zero/negative division

    # If min_ral is also 0 or negative, use a default scale factor
    if scale_factor <= 0: scale_factor = 1000 # Default scale if range/min is non-positive

    # Simple scaled inverse distance for scores > 0
    score = 1 / (distance / scale_factor + 1)

    return score

df_features['salary_fit_score'] = df_features.apply(
    lambda row: calculate_salary_fit(row['Expected Ral'], row['Minimum Ral'], row['Ral Maximum']),
    axis=1
)


# --- 4. Feature: Profile NLP Similarity ---
# Using Sentence Transformers to embed candidate summary and job profile text.

# Choose a pre-trained model
model_name = 'all-MiniLM-L6-v2' # A good balance of speed and performance
try:
    model = SentenceTransformer(model_name)
    nlp_model_loaded = True
except Exception as e:
    print(f"Error loading SentenceTransformer model {model_name}: {e}")
    print("Profile NLP similarity will be set to NaN.")
    nlp_model_loaded = False
    model = None # Set model to None if loading fails


if nlp_model_loaded:
    # Function to create a text summary for the candidate
    def create_candidate_text(row):
        # Combine relevant candidate features into a descriptive string
        parts = []
        # Include columns relevant to candidate's background/profile
        candidate_cols = ['Study area', 'Sector', 'Last Role'] # Add other relevant columns if needed
        for col in candidate_cols:
            if pd.notna(row[col]): parts.append(f"{col.replace('_', ' ')}: {row[col]}")
        # Add parsed experience if it provides useful context as text
        if pd.notna(row['Years Experience_parsed']): parts.append(f"Experience: {row['Years Experience_parsed']} years")
        return ". ".join(parts) if parts else ""

    # Function to combine relevant job text
    def create_job_text(row):
        parts = []
         # Include columns relevant to job requirements
        job_cols = ['Job Title Hiring', 'Job Description', 'Candidate Profile', 'Study Area.1'] # Add other relevant columns
        for col in job_cols:
             if pd.notna(row[col]): parts.append(f"{col.replace('_', ' ')}: {row[col]}")
        # Add parsed required experience as text
        if pd.notna(row['Years Experience.1_parsed']): parts.append(f"Required Experience: {row['Years Experience.1_parsed']} years")
        return ". ".join(parts) if parts else ""


    # Create text columns on the features dataframe
    df_features['candidate_text'] = df_features.apply(create_candidate_text, axis=1)
    df_features['job_text'] = df_features.apply(create_job_text, axis=1)

    # Generate embeddings (handle empty strings or NaNs in text columns)
    candidate_texts = df_features['candidate_text'].tolist()
    job_texts = df_features['job_text'].tolist()

    try:
        # Process only non-empty strings for embeddings
        # Map original index to the index in the filtered list
        valid_candidate_indices = [i for i, txt in enumerate(candidate_texts) if txt]
        valid_job_indices = [i for i, txt in enumerate(job_texts) if txt]

        valid_candidate_texts_filtered = [candidate_texts[i] for i in valid_candidate_indices]
        valid_job_texts_filtered = [job_texts[i] for i in valid_job_indices]


        if valid_candidate_texts_filtered and valid_job_texts_filtered:
            # Ensure model is loaded before encoding
            if model is None:
                 raise ValueError("NLP model is not loaded.")

            candidate_embeddings = model.encode(valid_candidate_texts_filtered, show_progress_bar=False)
            job_embeddings = model.encode(valid_job_texts_filtered, show_progress_bar=False)

            # Calculate cosine similarity and map back to the original index
            similarity_scores = [np.nan] * len(df_features) # Initialize with NaN or 0
            
            # Create mappings from filtered index back to original index
            candidate_idx_map = {i_filtered: i_orig for i_filtered, i_orig in enumerate(valid_candidate_indices)}
            job_idx_map = {i_filtered: i_orig for i_filtered, i_orig in enumerate(valid_job_indices)}

            # Calculate similarity only for rows where both texts were valid
            # Find original indices that are in both valid lists
            common_indices = list(set(valid_candidate_indices) & set(valid_job_indices))
            
            # Create mappings from original index to filtered index for common indices
            original_to_filtered_cand = {i_orig: i_filtered for i_filtered, i_orig in enumerate(valid_candidate_indices)}
            original_to_filtered_job = {i_filtered: i_orig for i_filtered, i_orig in enumerate(valid_job_indices)}
             # Need a map for job filtered index too, if the lists aren't the same length
            original_to_filtered_job = {i_orig: i_filtered for i_filtered, i_orig in enumerate(valid_job_indices)}


            for i_orig in common_indices:
                 i_filtered_cand = original_to_filtered_cand[i_orig]
                 i_filtered_job = original_to_filtered_job[i_orig] # Corrected mapping here

                 cand_emb = candidate_embeddings[i_filtered_cand].reshape(1, -1)
                 job_emb = job_embeddings[i_filtered_job].reshape(1, -1)
                 score = cosine_similarity(cand_emb, job_emb)[0][0]
                 similarity_scores[i_orig] = score # Assign score to the original index

            # Fill NaN for rows where one or both texts were empty
            for i in range(len(df_features)):
                 if i not in common_indices:
                      # Decide default score for missing text - 0 or NaN
                      similarity_scores[i] = 0.0 # Assuming 0 similarity if text is missing


            df_features['profile_nlp_similarity'] = similarity_scores

        else:
            # If no valid text pairs at all, set all scores to 0 or NaN
            df_features['profile_nlp_similarity'] = 0.0 # Or np.nan

    except Exception as e:
        print(f"Error during embedding or similarity calculation: {e}")
        df_features['profile_nlp_similarity'] = np.nan # Set to NaN if NLP fails

    # Drop temporary text columns
    df_features = df_features.drop(columns=['candidate_text', 'job_text'])

else:
    df_features['profile_nlp_similarity'] = np.nan # Set to NaN if model loading failed


# --- 5. Feature: Geographical Distance ---
# Requires a geocoding service (Nominatim is free but has rate limits and usage policies)
# Define the HQ location (example: a specific address in Bologna)
hq_location_str = "Via dei Fornari 12, Bologna, Italy"
geolocator = Nominatim(user_agent="geopy_distance_calculator_v4") # Add a unique user agent

# Dictionary to cache geocoding results to avoid redundant calls
geocoding_cache = {}

def get_coordinates_cached(location_str):
    if pd.isna(location_str) or location_str.strip() == "":
        return None
    if location_str in geocoding_cache:
        return geocoding_cache[location_str]

    try:
        # Use try-except for geocoding failures and rate limiting
        # Increased timeout for robustness
        location = geolocator.geocode(location_str, timeout=10)
        if location:
            coords = (location.latitude, location.longitude)
            geocoding_cache[location_str] = coords
             # Add a small delay to respect Nominatim rate limits *only on success*
            time.sleep(1.1) # Slightly more than 1 second
            return coords
        else:
            geocoding_cache[location_str] = None # Cache failure
            time.sleep(0.5) # Shorter wait on failure but still wait
            return None
    except Exception as e:
        # print(f"Geocoding failed for '{location_str}': {e}") # Keep this commented unless debugging geocoding
        geocoding_cache[location_str] = None # Cache failure
        time.sleep(1.1) # Still wait to avoid hitting the service too hard on errors
        return None

# Get HQ coordinates once
hq_coords = get_coordinates_cached(hq_location_str)
# print(f"\nHQ Coordinates for '{hq_location_str}': {hq_coords}") # Commented for cleaner output

def calculate_distance_to_hq(row, hq_coords):
    if hq_coords is None:
        return np.nan # Cannot calculate if HQ coords are unknown

    # Prefer city, then province/region, then country for geocoding
    location_parts = []
    # Order matters for geocoding accuracy, start from most specific
    if pd.notna(row['Residence Italian City']) and str(row['Residence Italian City']).strip():
        location_parts.append(str(row['Residence Italian City']).strip())
    if pd.notna(row['Residence Italian Province']) and str(row['Residence Italian Province']).strip():
        location_parts.append(f"{str(row['Residence Italian Province']).strip()} Province") # Add ' Province' for clarity
    if pd.notna(row['Residence Italian Region']) and str(row['Residence Italian Region']).strip():
         location_parts.append(f"{str(row['Residence Italian Region']).strip()} Region") # Add ' Region' for clarity
    if pd.notna(row['Residence Country']) and str(row['Residence Country']).strip():
        location_parts.append(str(row['Residence Country']).strip())

    # If it's an Italian residence, prioritize Italian structure before adding country
    if row.get('Italian Residence', False) and len(location_parts) > 1:
         # Reorder to City, Province, Region, Country if Italian
         italian_parts = []
         city = [p for p in location_parts if 'City' not in p and 'Province' not in p and 'Region' not in p and 'Country' not in p] # Simple city name
         province = [p for p in location_parts if 'Province' in p]
         region = [p for p in location_parts if 'Region' in p]
         country = [p for p in location_parts if 'Country' in p] # Country might be 'Italy'

         if city: italian_parts.extend(city)
         if province: italian_parts.extend(province)
         if region: italian_parts.extend(region)
         if country: italian_parts.extend(country)
         # Fallback if reordering fails or is empty
         candidate_location_str = ", ".join(italian_parts) if italian_parts else ", ".join(location_parts)

    else:
         # For non-Italian or incomplete Italian addresses, just join in order
         candidate_location_str = ", ".join(location_parts)


    if not candidate_location_str:
        return np.nan # No location info for candidate

    candidate_coords = get_coordinates_cached(candidate_location_str)

    if candidate_coords:
        try:
            # Calculate geodesic distance in kilometers
            distance_km = geodesic(candidate_coords, hq_coords).km
            return distance_km
        except Exception as e:
            # print(f"Distance calculation failed for coords {candidate_coords} and {hq_coords}: {e}") # Debugging distance
            return np.nan
    else:
        return np.nan # Could not geocode candidate location

# Apply distance calculation row by row
# This can be slow due to geocoding API calls and delays
# Consider parallelizing or using a faster geocoding service for large datasets
print("\n--- Calculating geographical distances... This may take some time depending on the number of rows and API rate limits. ---")
df_features['residence_hq_distance_km'] = df_features.apply(
    lambda row: calculate_distance_to_hq(row, hq_coords),
    axis=1
)
print("--- Geographical distance calculation finished. ---")


# Create a proximity score from distance
df_features['residence_hq_proximity_score'] = df_features['residence_hq_distance_km'].apply(
    lambda x: 1 / (x + 1) if pd.notna(x) and x >= 0 else (1.0 if pd.notna(x) and x < 0 else np.nan) # Inverse distance score (handle potential negative distance though unlikely)
)


# --- 6. Feature: Overall Match Score ---
# Combine relevant scores. This is a simple example, weighting can be adjusted.

# Select scores to combine
# Include the new features and potentially scaled existing performance scores
score_columns_for_overall = [
    'experience_match_score',
    'salary_fit_score',
    'profile_nlp_similarity',
    'residence_hq_proximity_score'
]

# Scale 'Overall' performance score to 0-1 range if needed (assuming max is 5)
# Ensure 'Overall' is numeric before scaling (done earlier in cleaning)
df_features['Overall_scaled'] = df_features['Overall'] / 5.0
score_columns_for_overall.append('Overall_scaled')

# Simple average, ignoring NaNs
df_features['overall_match_score'] = df_features[score_columns_for_overall].mean(axis=1)


# --- 7. Drop Original Columns Used for Feature Creation ---
# Define the list of original columns that were used to derive the new features
# Drop the original string/object columns and the intermediate parsed/scaled columns
columns_to_drop_refined = [
    'Study area', 'Sector', 'Last Role', # Used for candidate text
    'Years Experience', # Original string column
    'Current Ral', 'Expected Ral', 'Minimum Ral', 'Ral Maximum', # Original RAL columns (if parsed later)
    'Job Title Hiring', 'Job Description', 'Candidate Profile', 'Study Area.1', # Used for job text
    'Years Experience.1', # Original string column
    'Residence Country', 'Residence Italian Region',
    'Residence Italian Province', 'Residence Italian City', 'Akkodis headquarters', # Used for geocoding
    'Overall_scaled', # Intermediate scaled column
    'Years Experience_parsed', 'Years Experience.1_parsed' # Intermediate parsed numeric columns (keep if needed)
]

# Decide which original/intermediate columns to KEEP
columns_to_keep = [
    '# TAG', 'Italian Residence', 'European Residence',
    'Overall', 'Technical Skills', 'Standing/Position',
    'Comunication', 'Maturity', 'Dynamism', 'Mobility', 'English',
    'experience_match_score', 'salary_fit_score', 'profile_nlp_similarity',
    'residence_hq_distance_km', 'residence_hq_proximity_score', 'overall_match_score', 'Hired'
    # Optionally, keep the parsed numeric experience/RALs if useful
    # 'Years Experience_parsed', 'Years Experience.1_parsed',
    # 'Current Ral', 'Expected Ral', 'Minimum Ral', 'Ral Maximum'
]

# Get the list of columns to drop by finding columns in df_features that are NOT in columns_to_keep
all_df_features_cols = set(df_features.columns)
columns_to_drop_actual = list(all_df_features_cols - set(columns_to_keep))


# Ensure columns exist before trying to drop them
columns_to_drop_actual = [col for col in columns_to_drop_actual if col in df_features.columns]

df_features = df_features.drop(columns=columns_to_drop_actual)


# --- Final Output ---
print("\n--- Original DataFrame remains unchanged ---")
print(df.head())
print("\n--- Original DataFrame Info (should be same as before conversion) ---")
df.info()


print("\n--- Processed DataFrame with New Float Features (first 5 rows) ---")
# Display the processed dataframe - should have new columns and fewer old ones
print(df_features.head())

print("\n--- Info on Processed DataFrame ---")
df_features.info()

# Optional: Display the columns in the processed DataFrame
print("\n--- Columns in Processed DataFrame ---")
print(df_features.columns.tolist())
 



--- DataFrame after parsing and converting numeric columns (first 5 rows) ---
  Years Experience  Years Experience_parsed Years Experience.1  \
0              [0]                      0.0                NaN   
1              [0]                      0.0                NaN   
2            [1-3]                      2.0              [1-3]   
3            [3-5]                      4.0                NaN   
4              [0]                      0.0                NaN   

   Years Experience.1_parsed  Current Ral  Expected Ral  Minimum Ral  \
0                        NaN          NaN           NaN          NaN   
1                        NaN          NaN           NaN          NaN   
2                        2.0          NaN           NaN          NaN   
3                        NaN          NaN           NaN          NaN   
4                        NaN          NaN           NaN          NaN   

   Ral Maximum  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4     

In [72]:
import numpy as np
# Import TF-IDF Vectorizer and cosine_similarity from sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import time # To handle geocoding rate limits
import warnings # To manage potential warnings
import re # Import regex for parsing strings


df_features = final_df.copy()

# --- Data Cleaning: Convert specific columns to numeric ---

# Custom parsing function for experience strings like '[0-1]', '[+10]', '[3-5] | [1-3]'
def parse_experience_string(exp_str):
    if pd.isna(exp_str):
        return np.nan
    if not isinstance(exp_str, str):
         # If it's already a number (e.g., int/float not converted to str yet), return it
         return float(exp_str)

    exp_str = exp_str.strip()
    if not exp_str:
        return np.nan

    # Handle combined formats like '[3-5] | [1-3]'
    parts = exp_str.split('|')
    values = []

    for part in parts:
        part = part.strip().replace('[', '').replace(']', '') # Remove brackets
        if not part: continue

        try:
            if '-' in part:
                if part.startswith('+'): # Handle '+10' case explicitly if it appears as '+10'
                     value = float(part.replace('+', '')) # Treat +10 as 10
                else:
                    low, high = map(float, part.split('-'))
                    value = (low + high) / 2.0 # Midpoint of range
            elif part.startswith('+'): # Handle '+10' when it's just '+10' after stripping []
                 value = float(part.replace('+', '')) # Treat +10 as 10
            else:
                value = float(part) # Handle single numbers like '0', '5', '10'

            values.append(value)
        except ValueError:
            # print(f"Warning: Could not parse experience string part '{part}' from '{exp_str}'")
            continue # Ignore parts that cannot be parsed

    if values:
        # For multiple ranges/values separated by '|', take the maximum experience
        return max(values)
    else:
        return np.nan # Return NaN if no parts could be parsed

# Apply custom parsing to experience columns
df_features['Years Experience_parsed'] = df_features['Years Experience'].apply(parse_experience_string)
df_features['Years Experience.1_parsed'] = df_features['Years Experience.1'].apply(parse_experience_string)

# Convert RAL columns and Overall to numeric, coercing errors
ral_overall_cols = ['Current Ral', 'Expected Ral', 'Minimum Ral', 'Ral Maximum', 'Overall']
for col in ral_overall_cols:
     # Simple conversion. If 'k' format exists, this will turn it to NaN.
     # A more complex parser would be needed for 'k'. Sticking to simple numeric/NaN for now.
     df_features[col] = pd.to_numeric(df_features[col], errors='coerce')


print("\n--- DataFrame after parsing and converting numeric columns (first 5 rows) ---")
print(df_features[['Years Experience', 'Years Experience_parsed', 'Years Experience.1', 'Years Experience.1_parsed', 'Current Ral', 'Expected Ral', 'Minimum Ral', 'Ral Maximum']].head())
print("\n--- DataFrame Info after parsing and converting numeric columns ---")
df_features[['Years Experience_parsed', 'Years Experience.1_parsed'] + ral_overall_cols].info()


# --- 2. Feature: Experience Match Score ---
# Simple inverse of difference, scaled. Smaller difference = higher score.
# Now use the parsed numeric columns
def calculate_experience_match(candidate_exp_parsed, job_req_exp_parsed):
    # pd.isna handles both np.nan and None
    if pd.isna(candidate_exp_parsed) or pd.isna(job_req_exp_parsed):
        return np.nan
    diff = abs(candidate_exp_parsed - job_req_exp_parsed)
    # Score decreases as difference increases, never 0 unless diff is infinite
    return 1 / (diff + 1)

df_features['experience_match_score'] = df_features.apply(
    lambda row: calculate_experience_match(row['Years Experience_parsed'], row['Years Experience.1_parsed']),
    axis=1
)


# --- 3. Feature: Salary Fit Score ---
# Score based on whether expected RAL is within the min/max range.
# Using the converted RAL columns
def calculate_salary_fit(expected_ral, min_ral, max_ral):
    # pd.isna handles both np.nan and None
    if pd.isna(expected_ral) or pd.isna(min_ral) or pd.isna(max_ral):
        return np.nan

    if min_ral > max_ral: # Handle illogical ranges
         return np.nan

    if expected_ral >= min_ral and expected_ral <= max_ral:
        return 1.0 # Perfect fit

    # If outside the range, score decreases with distance from the range
    if expected_ral < min_ral:
        distance = min_ral - expected_ral
    else: # expected_ral > max_ral
        distance = expected_ral - max_ral

    # Scale the distance (adjust denominator based on expected RAL scale)
    # Using the range size + min_ral as a scaling factor
    range_size = max_ral - min_ral
    scale_factor = range_size if range_size > 0 else min_ral # Avoid zero/negative division

    # If min_ral is also 0 or negative, use a default scale factor
    if scale_factor <= 0: scale_factor = 1000 # Default scale if range/min is non-positive

    # Simple scaled inverse distance for scores > 0
    score = 1 / (distance / scale_factor + 1)

    return score

df_features['salary_fit_score'] = df_features.apply(
    lambda row: calculate_salary_fit(row['Expected Ral'], row['Minimum Ral'], row['Ral Maximum']),
    axis=1
)


# --- 4. Feature: Profile NLP Similarity (using TF-IDF) ---

# Function to create a text summary for the candidate (same as before)
def create_candidate_text(row):
    # Combine relevant candidate features into a descriptive string
    parts = []
    # Include columns relevant to candidate's background/profile
    candidate_cols = ['Study area', 'Sector', 'Last Role'] # Corrected 'Study area' to 'Study Area'
    for col in candidate_cols:
        if pd.notna(row[col]): parts.append(f"{col.replace('_', ' ')}: {row[col]}")
    # Add parsed experience if it provides useful context as text
    if pd.notna(row['Years Experience_parsed']): parts.append(f"Experience: {row['Years Experience_parsed']} years")
    return ". ".join(parts) if parts else ""

# Function to combine relevant job text (same as before)
def create_job_text(row):
    parts = []
     # Include columns relevant to job requirements
    job_cols = ['Job Title Hiring', 'Job Description', 'Candidate Profile', 'Study Area.1'] # Add other relevant columns
    for col in job_cols:
         if pd.notna(row[col]): parts.append(f"{col.replace('_', ' ')}: {row[col]}")
    # Add parsed required experience as text
    if pd.notna(row['Years Experience.1_parsed']): parts.append(f"Required Experience: {row['Years Experience.1_parsed']} years")
    return ". ".join(parts) if parts else ""

# Create text columns on the features dataframe
df_features['candidate_text'] = df_features.apply(create_candidate_text, axis=1).fillna("") # Fill NaN text with empty string
df_features['job_text'] = df_features.apply(create_job_text, axis=1).fillna("") # Fill NaN text with empty string

# Combine all texts to build the TF-IDF vocabulary
all_texts = df_features['candidate_text'].tolist() + df_features['job_text'].tolist()

# Initialize TF-IDF Vectorizer
# You can add parameters like stop_words='english', ngram_range=(1, 2), max_features=...
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=1000) # Limit features to 1000 for demonstration

# Fit the vectorizer on all texts to build the vocabulary and IDF
tfidf_vectorizer.fit(all_texts)

# Transform candidate and job texts into TF-IDF vectors
candidate_tfidf = tfidf_vectorizer.transform(df_features['candidate_text'])
job_tfidf = tfidf_vectorizer.transform(df_features['job_text'])

# Calculate cosine similarity between candidate and job vectors
# cosine_similarity returns a matrix where element (i, j) is similarity between vector i and vector j.
# We want the similarity between vector i (candidate i) and vector i (job i).
# This is the diagonal of the similarity matrix, or simply the dot product if vectors are L2-normalized (which TfidfVectorizer does).
# Using pairwise.cosine_similarity is straightforward:
similarity_matrix = cosine_similarity(candidate_tfidf, job_tfidf)

# The similarity score for each pair (candidate_i, job_i) is on the diagonal
profile_nlp_similarity_scores = similarity_matrix.diagonal()

df_features['profile_nlp_similarity'] = profile_nlp_similarity_scores

# Drop temporary text columns
df_features = df_features.drop(columns=['candidate_text', 'job_text'])


# --- 5. Feature: Geographical Distance (Same as before) ---
# Requires a geocoding service (Nominatim is free but has rate limits and usage policies)
# Define the HQ location (example: a specific address in Bologna)
hq_location_str = "Via dei Fornari 12, Bologna, Italy"
geolocator = Nominatim(user_agent="geopy_distance_calculator_v5") # Add a unique user agent

# Dictionary to cache geocoding results to avoid redundant calls
geocoding_cache = {}

def get_coordinates_cached(location_str):
    if pd.isna(location_str) or location_str.strip() == "":
        return None
    if location_str in geocoding_cache:
        return geocoding_cache[location_str]

    try:
        # Use try-except for geocoding failures and rate limiting
        # Increased timeout for robustness
        location = geolocator.geocode(location_str, timeout=10)
        if location:
            coords = (location.latitude, location.longitude)
            geocoding_cache[location_str] = coords
             # Add a small delay to respect Nominatim rate limits *only on success*
            time.sleep(1.1) # Slightly more than 1 second
            return coords
        else:
            geocoding_cache[location_str] = None # Cache failure
            time.sleep(0.5) # Shorter wait on failure but still wait
            return None
    except Exception as e:
        # print(f"Geocoding failed for '{location_str}': {e}") # Keep this commented unless debugging geocoding
        geocoding_cache[location_str] = None # Cache failure
        time.sleep(1.1) # Still wait to avoid hitting the service too hard on errors
        return None

# Get HQ coordinates once
hq_coords = get_coordinates_cached(hq_location_str)
# print(f"\nHQ Coordinates for '{hq_location_str}': {hq_coords}") # Commented for cleaner output

def calculate_distance_to_hq(row, hq_coords):
    if hq_coords is None:
        return np.nan # Cannot calculate if HQ coords are unknown

    # Prefer city, then province/region, then country for geocoding
    location_parts = []
    # Order matters for geocoding accuracy, start from most specific
    if pd.notna(row['Residence Italian City']) and str(row['Residence Italian City']).strip():
        location_parts.append(str(row['Residence Italian City']).strip())
    if pd.notna(row['Residence Italian Province']) and str(row['Residence Italian Province']).strip():
        location_parts.append(f"{str(row['Residence Italian Province']).strip()} Province") # Add ' Province' for clarity
    if pd.notna(row['Residence Italian Region']) and str(row['Residence Italian Region']).strip():
         location_parts.append(f"{str(row['Residence Italian Region']).strip()} Region") # Add ' Region' for clarity
    if pd.notna(row['Residence Country']) and str(row['Residence Country']).strip():
        location_parts.append(str(row['Residence Country']).strip())

    # If it's an Italian residence, prioritize Italian structure before adding country
    if row.get('Italian Residence', False) and len(location_parts) > 1:
         # Reorder to City, Province, Region, Country if Italian
         italian_parts = []
         city = [p for p in location_parts if 'City' not in p and 'Province' not in p and 'Region' not in p and 'Country' not in p] # Simple city name
         province = [p for p in location_parts if 'Province' in p]
         region = [p for p in location_parts if 'Region' in p]
         country = [p for p in location_parts if 'Country' in p] # Country might be 'Italy'

         if city: italian_parts.extend(city)
         if province: italian_parts.extend(province)
         if region: italian_parts.extend(region)
         if country: italian_parts.extend(country)
         # Fallback if reordering fails or is empty
         candidate_location_str = ", ".join(italian_parts) if italian_parts else ", ".join(location_parts)

    else:
         # For non-Italian or incomplete Italian addresses, just join in order
         candidate_location_str = ", ".join(location_parts)


    if not candidate_location_str:
        return np.nan # No location info for candidate

    candidate_coords = get_coordinates_cached(candidate_location_str)

    if candidate_coords:
        try:
            # Calculate geodesic distance in kilometers
            distance_km = geodesic(candidate_coords, hq_coords).km
            return distance_km
        except Exception as e:
            # print(f"Distance calculation failed for coords {candidate_coords} and {hq_coords}: {e}") # Debugging distance
            return np.nan
    else:
        return np.nan # Could not geocode candidate location

# Apply distance calculation row by row
# This can be slow due to geocoding API calls and delays
# Consider parallelizing or using a faster geocoding service for large datasets
print("\n--- Calculating geographical distances... This may take some time depending on the number of rows and API rate limits. ---")
df_features['residence_hq_distance_km'] = df_features.apply(
    lambda row: calculate_distance_to_hq(row, hq_coords),
    axis=1
)
print("--- Geographical distance calculation finished. ---")


# Create a proximity score from distance
df_features['residence_hq_proximity_score'] = df_features['residence_hq_distance_km'].apply(
    lambda x: 1 / (x + 1) if pd.notna(x) and x >= 0 else (1.0 if pd.notna(x) and x < 0 else np.nan) # Inverse distance score (handle potential negative distance though unlikely)
)


# --- 6. Feature: Overall Match Score ---
# Combine relevant scores. This is a simple example, weighting can be adjusted.

# Select scores to combine
# Include the new features and potentially scaled existing performance scores
score_columns_for_overall = [
    'experience_match_score',
    'salary_fit_score',
    'profile_nlp_similarity',
    'residence_hq_proximity_score'
]

# Scale 'Overall' performance score to 0-1 range if needed (assuming max is 5)
# Ensure 'Overall' is numeric before scaling (done earlier in cleaning)
df_features['Overall_scaled'] = df_features['Overall'] / 5.0
score_columns_for_overall.append('Overall_scaled')

# Simple average, ignoring NaNs
df_features['overall_match_score'] = df_features[score_columns_for_overall].mean(axis=1)


# --- 7. Drop Original Columns Used for Feature Creation ---
# Define the list of original columns that were used to derive the new features
# Drop the original string/object columns and the intermediate parsed/scaled columns
# Keeping original numeric-like columns after conversion for potential use as features
columns_to_drop_refined = [
    'Study Area', 'Sector', 'Last Role', # Used for candidate text
    'Years Experience', # Original string column
    # 'Current Ral', 'Expected Ral', 'Minimum Ral', 'Ral Maximum', # Keeping these after conversion
    'Job Title Hiring', 'Job Description', 'Candidate Profile', 'Study Area.1', # Used for job text
    'Years Experience.1', # Original string column
    'Residence Country', 'Residence Italian Region',
    'Residence Italian Province', 'Residence Italian City', 'Akkodis headquarters', # Used for geocoding
    'Overall_scaled', # Intermediate scaled column
    'Years Experience_parsed', 'Years Experience.1_parsed' # Intermediate parsed numeric columns (keeping original numeric-like ones instead)
]

# Decide which original/intermediate columns to KEEP
# Explicitly list the columns that should remain in the final df_features
columns_to_keep = [
    '# TAG', 'Italian Residence', 'European Residence', # Identifiers/Booleans
    'Overall', 'Technical Skills', 'Standing/Position', # Original Scores
    'Comunication', 'Maturity', 'Dynamism', 'Mobility', 'English', # Original Scores
    'Current Ral', 'Expected Ral', 'Minimum Ral', 'Ral Maximum', # Original RALs (now numeric/NaN)
    'Years Experience_parsed', 'Years Experience.1_parsed', # Keeping parsed years experience
    'experience_match_score', 'salary_fit_score', # New Scores
    'profile_nlp_similarity', # New NLP Score
    'residence_hq_distance_km', 'residence_hq_proximity_score', # New Geo Scores
    'overall_match_score' # New Combined Score
    # Need to add 'Hired' back in if you want it in the final features df before ML
    # For training, we separate X and y, so it doesn't *have* to be in df_features permanently
]


df_features['Hired_cleaned'] = (df_features['Hired'] == True)
df_features['Hired'] = df_features['Hired_cleaned']
df_features = df_features.drop(columns=['Hired_cleaned'])
print("\n'Hired' column cleaned and added to df_features.")


# Add 'Hired' to the list of columns to keep
columns_to_keep.append('Hired')


# Get the list of columns to drop by finding columns in df_features that are NOT in columns_to_keep
all_df_features_cols = set(df_features.columns)
columns_to_drop_actual = list(all_df_features_cols - set(columns_to_keep))

# Ensure columns exist before trying to drop them
columns_to_drop_actual = [col for col in columns_to_drop_actual if col in df_features.columns]

# Drop the columns
df_features = df_features.drop(columns=columns_to_drop_actual)



--- DataFrame after parsing and converting numeric columns (first 5 rows) ---
  Years Experience  Years Experience_parsed Years Experience.1  \
0              [0]                      0.0                NaN   
1              [0]                      0.0                NaN   
2            [1-3]                      2.0              [1-3]   
3            [3-5]                      4.0                NaN   
4              [0]                      0.0                NaN   

   Years Experience.1_parsed  Current Ral  Expected Ral  Minimum Ral  \
0                        NaN          NaN           NaN          NaN   
1                        NaN          NaN           NaN          NaN   
2                        2.0          NaN           NaN          NaN   
3                        NaN          NaN           NaN          NaN   
4                        NaN          NaN           NaN          NaN   

   Ral Maximum  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4     

In [73]:
final_df.isna().mean() * 100

ID                             0.000000
Candidate State                0.000000
Age Range                      0.000000
Residence                      0.000000
Sex                            0.000000
Protected category             0.000000
TAG                           47.120843
Study area                     0.000000
Study Title                    0.000000
Years Experience               0.000000
Sector                         9.914842
Last Role                     10.036496
Year of insertion              0.000000
Recruitment Request           91.423358
Assumption Headquarters       89.963504
Job Family Hiring             89.963504
Job Title Hiring              90.612328
Overall                       54.075426
Job Description               91.362530
Candidate Profile             91.443633
Years Experience.1            91.342255
Minimum Ral                   95.133820
Ral Maximum                   93.714517
Study Level                   91.342255
Study Area.1                  91.342255


In [74]:
df_features.isna().mean() * 100

Overall                         100.000000
Minimum Ral                     100.000000
Ral Maximum                     100.000000
Current Ral                     100.000000
Expected Ral                    100.000000
Technical Skills                 54.197080
Standing/Position                54.197080
Comunication                     54.197080
Maturity                         54.197080
Dynamism                         54.217356
Mobility                         54.217356
English                          54.278183
Hired                             0.000000
Italian Residence                 0.000000
European Residence                0.000000
Years Experience_parsed           0.000000
Years Experience.1_parsed        91.342255
experience_match_score           91.342255
salary_fit_score                100.000000
profile_nlp_similarity            0.000000
residence_hq_distance_km        100.000000
residence_hq_proximity_score    100.000000
overall_match_score               0.000000
dtype: floa

In [75]:
df

,ID,Candidate State,Age Range,Residence,Sex,Protected category,TAG,Study area,Study Title,Years Experience,Sector,Last Role,Year of insertion,Recruitment Request,Assumption Headquarters,Job Family Hiring,Job Title Hiring,event_type__val,event_feedback,Overall,Job Description,Candidate Profile,Years Experience.1,Minimum Ral,Ral Maximum,Study Level,Study Area.1,Akkodis headquarters,Current Ral,Expected Ral,Technical Skills,Standing/Position,Comunication,Maturity,Dynamism,Mobility,English,Hired,Residence Country,Residence Italian Region,Residence Italian Province,Residence Italian City,Italian Residence,European Residence
0,41,first contact,31 - 35 years,PERCH DOSIMO » Cremona ~ Lombardy,Male,False,NaN,industrial engineering,Five-year degree,[1-3],Others,Business Analyst Consultant,[2021],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,ITALY,PERCH DOSIMO,CREMONA,LOMBARDY,True,True
1,47,first contact,36 - 40 years,GERMANY » (STATE) ~ (OVERSEAS),Male,False,NaN,Electrical Engineering,Five-year degree,[+10],Energy,ET Solutions AG,[2022],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,GERMANY,None,None,None,False,True
2,84,first contact,26 - 30 years,BOLOGNA » Bologna ~ Emilia Romagna,Male,False,NaN,Civil/Civil and Environmental Engineering,Five-year degree,[0-1],Others,Civil Engineer - Internship at Studio,[2022],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,ITALY,BOLOGNA,BOLOGNA,EMILIA ROMAGNA,True,True
3,114,first contact,26 - 30 years,TURIN » Turin ~ Piedmont,Female,False,"-, 3D PRINTING PREFORM SOFTWARE; PYTHON; ANSYS...",Biomedical Engineering,Five-year degree,[0],NaN,NaN,[2022],NaN,NaN,NaN,NaN,cv request,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,ITALY,TURIN,TURIN,PIEDMONT,True,True
4,114,first contact,26 - 30 years,TURIN » Turin ~ Piedmont,Female,False,"-, 3D PRINTING PREFORM SOFTWARE; PYTHON; ANSYS...",Biomedical Engineering,Five-year degree,[0],NaN,NaN,[2022],NaN,NaN,NaN,NaN,contact note,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,ITALY,TURIN,TURIN,PIEDMONT,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9416,9912_2,first contact,26 - 30 years,ROME » Rome ~ Lazio,Male,False,PYTHON,computer engineering,Five-year degree,[0],Others,Fresh graduate,[2023],NaN,NaN,NaN,NaN,contact note,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,ITALY,ROME,ROME,LAZIO,True,True
9417,9967_3,in selection,26 - 30 years,TURIN » Turin ~ Piedmont,Male,False,.,Energy and Nuclear Engineering,Three-year degree,[0],Energy,electrical system designer,[2022],NaN,NaN,NaN,NaN,research association,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,ITALY,TURIN,TURIN,PIEDMONT,True,True
9418,9967_3,in selection,26 - 30 years,TURIN » Turin ~ Piedmont,Male,False,.,Energy and Nuclear Engineering,Three-year degree,[0],Energy,electrical system designer,[2022],NaN,NaN,NaN,NaN,contact note,NaN,~ 2 - Medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,2.0,2.0,3.0,1.0,None,ITALY,TURIN,TURIN,PIEDMONT,True,True
9419,9967_3,in selection,26 - 30 years,TURIN » Turin ~ Piedmont,Male,False,.,Energy and Nuclear Engineering,Three-year degree,[0],Energy,electrical system designer,[2022],NaN,NaN,NaN,NaN,technical interview,KO (language skills),~ 1 - Low,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,2.0,3.0,1.0,None,ITALY,TURIN,TURIN,PIEDMONT,True,True


In [76]:
df_features['Hired'] = (df_features['Hired'] == True)


In [77]:
df_features

,Overall,Minimum Ral,Ral Maximum,Current Ral,Expected Ral,Technical Skills,Standing/Position,Comunication,Maturity,Dynamism,Mobility,English,Hired,Italian Residence,European Residence,Years Experience_parsed,Years Experience.1_parsed,experience_match_score,salary_fit_score,profile_nlp_similarity,residence_hq_distance_km,residence_hq_proximity_score,overall_match_score
0,NaN,NaN,NaN,NaN,NaN,2.000000,2.000000,3.000000,3.000000,3.0,2.0,2.0,False,True,True,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000
1,NaN,NaN,NaN,NaN,NaN,2.000000,2.000000,2.000000,2.000000,3.0,3.0,3.0,False,True,True,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000
2,NaN,NaN,NaN,NaN,NaN,2.500000,3.000000,3.000000,3.000000,3.0,1.0,3.0,True,True,True,2.0,2.0,1.0,NaN,0.139742,NaN,NaN,0.569871
3,NaN,NaN,NaN,NaN,NaN,2.333333,2.333333,2.333333,1.666667,2.0,2.0,3.0,False,True,True,4.0,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000
4,NaN,NaN,NaN,NaN,NaN,2.500000,2.000000,2.500000,2.500000,2.5,3.0,3.0,True,True,True,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,10.0,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000
4928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000
4929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,10.0,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000
4930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000


In [ ]:
feature_columns = [
    'experience_match_score',
    'salary_fit_score',
    'profile_nlp_similarity',
    'residence_hq_distance_km', # Keep distance or proximity score, not both usually
    # 'residence_hq_proximity_score', # Using distance, drop proximity
    'overall_match_score',
    'Overall', # Keep the original score
    'Technical Skills', 'Standing/Position', 'Comunication',
    'Maturity', 'Dynamism', 'Mobility', 'English',
    'Italian Residence', 'European Residence' # Include boolean features
    # Include parsed RALs/Experience if they weren't dropped and you want them as features
    # 'Years Experience_parsed', 'Years Experience.1_parsed',
    # 'Current Ral', 'Expected Ral', 'Minimum Ral', 'Ral Maximum'
]
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report
# Ensure all selected feature columns exist in the DataFrame
feature_columns = [col for col in feature_columns if col in df_features.columns]

X = df_features[feature_columns]
y = df_features['Hired']

# Handle missing values in features (X)
# Use SimpleImputer to replace NaNs (e.g., with the mean of the column)
# It's important to fit the imputer ONLY on the training data to avoid data leakage
imputer = SimpleImputer(strategy='mean') # Or 'median', 'constant'

# --- Split Data into Training and Testing Sets ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y) # Stratify to maintain hire ratio

# Fit the imputer on the training data and transform both training and test data
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test) # Use the *same* imputer fitted on training data

# --- Train a Simple ML Model (Logistic Regression) ---
model = LogisticRegression(random_state=42)

# Train the model
print("\n--- Training Logistic Regression Model ---")
model.fit(X_train_imputed, y_train)
print("--- Model Training Complete ---")

# --- Make Predictions and Evaluate (Optional but good practice) ---
y_pred = model.predict(X_test_imputed)

print("\n--- Model Evaluation ---")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



--- Training Logistic Regression Model ---
--- Model Training Complete ---

--- Model Evaluation ---
Accuracy: 0.9724249797242498

Classification Report:
              precision    recall  f1-score   support

       False       0.97      1.00      0.98      1091
        True       1.00      0.76      0.86       142

    accuracy                           0.97      1233
   macro avg       0.98      0.88      0.92      1233
weighted avg       0.97      0.97      0.97      1233



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, f1_score

# Ensure all selected feature columns exist in the DataFrame
feature_columns = [col for col in feature_columns if col in df_features.columns]

X = df_features[feature_columns]
y = df_features['Hired']

# Handle missing values in features (X)
# Use SimpleImputer to replace NaNs (e.g., with the mean of the column)
# It's important to fit the imputer ONLY on the training data to avoid data leakage
imputer = SimpleImputer(strategy='mean')  # You can also use 'median' or 'most_frequent'

# --- Split Data into Training and Testing Sets ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y) # Stratify to maintain hire ratio

# Fit the imputer on the training data and transform both training and test data
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)  # Use the *same* imputer fitted on training data

# --- Train a Better ML Model (Random Forest or Gradient Boosting) ---

# Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=5, min_samples_split=5)
print("\n--- Training Random Forest Classifier ---")
rf_model.fit(X_train_imputed, y_train)
print("--- Random Forest Training Complete ---")

# Gradient Boosting Classifier
gb_model = GradientBoostingClassifier(random_state=42, n_estimators=100, learning_rate=0.1, max_depth=3)
print("\n--- Training Gradient Boosting Classifier ---")
gb_model.fit(X_train_imputed, y_train)
print("--- Gradient Boosting Training Complete ---")

# --- Make Predictions and Evaluate ---
# Predict with Random Forest
rf_y_pred = rf_model.predict(X_test_imputed)
print("\n--- Random Forest Model Evaluation ---")
rf_accuracy = accuracy_score(y_test, rf_y_pred)
rf_f1 = f1_score(y_test, rf_y_pred, average='binary')
print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"Random Forest F1-Score: {rf_f1}")
print("\nRandom Forest Classification Report:")
print(classification_report(y_test, rf_y_pred))

# Predict with Gradient Boosting
gb_y_pred = gb_model.predict(X_test_imputed)
print("\n--- Gradient Boosting Model Evaluation ---")
gb_accuracy = accuracy_score(y_test, gb_y_pred)
gb_f1 = f1_score(y_test, gb_y_pred, average='binary')
print(f"Gradient Boosting Accuracy: {gb_accuracy}")
print(f"Gradient Boosting F1-Score: {gb_f1}")
print("\nGradient Boosting Classification Report:")
print(classification_report(y_test, gb_y_pred))



--- Training Random Forest Classifier ---
--- Random Forest Training Complete ---

--- Training Gradient Boosting Classifier ---
--- Gradient Boosting Training Complete ---

--- Random Forest Model Evaluation ---
Random Forest Accuracy: 0.9772911597729116
Random Forest F1-Score: 0.890625

Random Forest Classification Report:
              precision    recall  f1-score   support

       False       0.97      1.00      0.99      1091
        True       1.00      0.80      0.89       142

    accuracy                           0.98      1233
   macro avg       0.99      0.90      0.94      1233
weighted avg       0.98      0.98      0.98      1233


--- Gradient Boosting Model Evaluation ---
Gradient Boosting Accuracy: 0.975669099756691
Gradient Boosting F1-Score: 0.8846153846153846

Gradient Boosting Classification Report:
              precision    recall  f1-score   support

       False       0.98      1.00      0.99      1091
        True       0.97      0.81      0.88       142

  

In [80]:
len(X_test), len(X_train)

(1233, 3699)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# Import HistGradientBoostingClassifier instead of LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
# No need to import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, f1_score

feature_columns = [col for col in feature_columns if col in df_features.columns]

X = df_features[feature_columns]
y = df_features['Hired']

# --- Split Data into Training and Testing Sets (Same as before) ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# *** IMPORTANT: We skip the imputation step here ***
# The algorithm will handle the NaNs directly


# --- Train a Simple ML Model (HistGradientBoostingClassifier) ---
# This model can handle NaN values in the input features X
model = HistGradientBoostingClassifier(random_state=42)

# Train the model - pass X_train directly (it may contain NaNs)
print("\n--- Training HistGradientBoostingClassifier Model ---")
model.fit(X_train, y_train)
print("--- Model Training Complete ---")

# --- Make Predictions and Evaluate ---
# Predict - pass X_test directly (it may contain NaNs)
y_pred = model.predict(X_test)

print("\n--- Model Evaluation ---")
print("Accuracy:", accuracy_score(y_test, y_pred))

# Calculate F1 Score specifically
f1 = f1_score(y_test, y_pred)
print("F1 Score (for True/Hired class):", f1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


--- Training HistGradientBoostingClassifier Model ---
--- Model Training Complete ---

--- Model Evaluation ---
Accuracy: 0.9708029197080292
F1 Score (for True/Hired class): 0.8676470588235294

Classification Report:
              precision    recall  f1-score   support

       False       0.98      0.99      0.98      1091
        True       0.91      0.83      0.87       142

    accuracy                           0.97      1233
   macro avg       0.94      0.91      0.93      1233
weighted avg       0.97      0.97      0.97      1233



In [ ]:
X_train

,experience_match_score,salary_fit_score,profile_nlp_similarity,residence_hq_distance_km,overall_match_score,Overall,Technical Skills,Standing/Position,Comunication,Maturity,Dynamism,Mobility,English,Italian Residence,European Residence
4410,NaN,NaN,0.0,NaN,0.0,NaN,2.0,2.0,3.0,2.0,2.0,1.0,3.0,True,True
1998,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True
4413,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True
4367,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True
462,NaN,NaN,0.0,NaN,0.0,NaN,2.5,2.0,2.0,2.5,3.0,3.0,3.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2489,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True
4710,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True
79,NaN,NaN,0.0,NaN,0.0,NaN,3.0,3.0,3.0,3.0,3.0,3.0,3.0,True,True
1447,NaN,NaN,0.0,NaN,0.0,NaN,1.0,2.0,1.5,2.0,2.0,3.0,2.5,True,True


In [ ]:
set(final_df['Standing/Position'])
# TAG, Study Area, Sector, Last Role,  Years Experience, Current Ral, Expected Ral
# 'Job Title Hiring',  'Job Description', 'Candidate Profile', 'Years Experience.1', 'Minimum Ral', 'Ral Maximum', 'Study Level', 'Study Area.1',
# 'Residence Country', 'Residence Italian Region', 'Residence Italian Province', 'Residence Italian City', 'Italian Residence', 'European Residence'
# 'Akkodis headquarters'
# 'Overall', 'Technical Skills', 'Standing/Position', 'Comunication', 'Maturity', 'Dynamism', 'Mobility', 'English',

{1.0,
 2.0,
 3.0,
 2.75,
 2.5,
 1.5,
 4.0,
 2.25,
 1.75,
 1.25,
 3.5,
 3.25,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,

In [ ]:
set(final_df['Recruitment Request'])

{' 1 Maintenance Leader -  (PHASE 1) - Nigeria',
 ' Document Controller',
 ' Infotainment System Engineer',
 ' Mechanical Engineer Maintenance Specialist ',
 ' Mission Operation Engineer',
 ' Scrum Master',
 ' Team Procurement - Milan',
 ' Truck Analyzer Technical Support Consulting',
 ' Wiring Harness Architecture Engineer',
 ' interns',
 '3 Software engineer – microservices design & development_senior (>5 years)',
 'Accountant',
 'Administrative employee',
 'Aeroelastic Analysis ',
 'Aerospace QA/PA activities',
 'Airworthiness Engineer',
 'Analog/digital HW design',
 'Analysis - Mechanics_Window',
 'Analysis-Mechanics_Structure',
 'Analytical chemistry',
 'Application Engineer',
 'Application System - Medical Devices',
 'Application development and maintenance',
 'Automotive Business Manager',
 'Benchmarking Automotive Engineer',
 'Braking system design -  (Catia V5) ',
 'Braking systems design -(Catia V5) 3 Resources ',
 'Business Analyst',
 'Business Manager',
 'Business Manager 2

In [85]:
set(final_df['Candidate Profile'])

{'(coming soon)',
 "(profiles with experience in sales are also suitable, even those coming from the supplier's purchasing office, it is important that they have experience in the industrial or oil and gas sector) Excellent English required",
 '-',
 "- Bachelor's degree preferably in Biomedical Engineering, Electronics or equivalent - Good English - Interest in testing in the automotive sector - Team working skills - Assembling test benches --> optional | Bachelor's degree preferably in Biomedical Engineering, Electronics or equivalent - Good English - Interest in testing in the automotive sector - Team working skills - Assembling test benches --> optional",
 '- Degree in Electronic Engineering, Computer Science, Telecommunications. - Advanced knowledge of Interlocking systems for lines in Italy and abroad - Advanced knowledge of signaling principle schemes - Knowledge of the V401 scheme for Italian applications - Experience in the application design process for the generation of logic

### o

In [86]:
df.columns

Index(['ID', 'Candidate State', 'Age Range', 'Residence', 'Sex',
       'Protected category', 'TAG', 'Study area', 'Study Title',
       'Years Experience', 'Sector', 'Last Role', 'Year of insertion',
       'Recruitment Request', 'Assumption Headquarters', 'Job Family Hiring',
       'Job Title Hiring', 'event_type__val', 'event_feedback', 'Overall',
       'Job Description', 'Candidate Profile', 'Years Experience.1',
       'Minimum Ral', 'Ral Maximum', 'Study Level', 'Study Area.1',
       'Akkodis headquarters', 'Current Ral', 'Expected Ral',
       'Technical Skills', 'Standing/Position', 'Comunication', 'Maturity',
       'Dynamism', 'Mobility', 'English', 'Hired', 'Residence Country',
       'Residence Italian Region', 'Residence Italian Province',
       'Residence Italian City', 'Italian Residence', 'European Residence'],
      dtype='object')

In [ ]:
set(df['Last Role'])

{'Vehicle System Engineer',
 'Agile Project Manager',
 'Technical Sales',
 'Researcher ',
 'Computer engineer',
 'Automation and Innovation Engineer',
 'Pharmacy assistant',
 'Curricular Internship',
 'Control software development for industrial painting robots',
 'Associate PV&V Engineer',
 'Senior Process Engineer, Technical Engineering & Construction',
 'CAE Engineer - project manager',
 'Software development',
 'Graphic re-designer of food packaging',
 'Computer Vision Developer',
 'technical testing manager',
 'Computer expert',
 'Naval Engineer',
 'Internship as CAE Engineer|',
 'Cashier Managing the opening and closing of the cash register as sole manager',
 'Android Developer',
 'Accounting clerk',
 'Food sector production manager',
 'graduate',
 'Suspension specialist',
 'Digital Integrator Engineer',
 'Engineer-Structural Analysis via FEM/Fluid Dynamic Analysis via CFD',
 'IT',
 'internship for thesis',
 'Coordinator at the secretariat',
 'Special Processes Engineer ',
 'Meta

In [88]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from difflib import get_close_matches

## 1. Missing Value Replacement ###
df.fillna('not specified', inplace=True)

### 2. Format numeric fields ###
num_cols = ['Current Ral', 'Expected Ral']
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')


### 5. Standardize Last Role ###
# Dummy role mapping (can be extended)
role_mapping = {
    'software dev': 'Software Developer',
    'developer': 'Software Developer',
    'data scientist': 'Data Scientist',
}
df['Last Role'] = df['Last Role'].str.lower().replace(role_mapping)

### 6. Handle Duplicates by ID ###
# Detect conflicting duplicate IDs
grouped = df.groupby('ID')
new_ids = {}
for name, group in grouped:
    if group['Sex'].nunique() > 1:
        for idx, row in group.iterrows():
            new_ids[idx] = f"{row['ID']}_{row['Year of insertion']}"
    else:
        for idx, row in group.iterrows():
            new_ids[idx] = row['ID']

df['New_ID'] = df.index.map(new_ids)

### 7. Sorting Categorical Variables ###
# Define custom sort orders
cat_order = {
    'Candidate State': ['Imported', 'First contact', 'In selection', 'QM', 'Vivier', 'Economic proposal', 'Hired'],
    'Age Range': ['< 20', '20 - 25', '26 - 30', '31 - 35', '36 - 40', '40 - 45', '> 45'],
    'Years Experience': ['not specified', '[0]', '[0-1]', '[1-3]', '[3-5]', '[5-7]', '[7-10]', '[+10]']
}

for col, order in cat_order.items():
    df[col] = pd.Categorical(df[col], categories=order, ordered=True)

df.sort_values(by=['Candidate State', 'Age Range', 'Years Experience'], inplace=True)

### 8. Filtering Candidates ###
# Remove early stage or ambiguous candidates
invalid_states = ['imported', 'first contact']
df = df[~df['Candidate State'].isin(invalid_states)]

# Filter based on event feedback
invalid_feedback = [
    'OK (other candidate)', 'KO (lost availability)', 'OK (hired)',
    'OK (waiting for departure)', 'KO (opportunity closed)', 'KO (retired)',
    'KO (ral)', 'KO (proposed renunciation)'
]
df = df[~df['event_feedback'].isin(invalid_feedback)]

# ### 9. Collapse Events: Aggregate Interview Scores ###
# score_cols = ['Technical Skills', 'Standing/Position', 'Comunication', 'Dynamism', 'Mobility', 'English']
# df_scores = df.groupby('New_ID')[score_cols].mean().reset_index()
# df_final = df.drop_duplicates('New_ID').merge(df_scores, on='New_ID', suffixes=('', '_avg'))

# ### 10. Final Dataset: Binary Labels (Hired = 1) ###
# df_final['Hired'] = (df_final['Candidate State'] == 'hired').astype(int)
df

,ID,Candidate State,Age Range,Residence,Sex,Protected category,TAG,Study area,Study Title,Years Experience,Sector,Last Role,Year of insertion,Recruitment Request,Assumption Headquarters,Job Family Hiring,Job Title Hiring,event_type__val,event_feedback,Overall,Job Description,Candidate Profile,Years Experience.1,Minimum Ral,Ral Maximum,Study Level,Study Area.1,Akkodis headquarters,Current Ral,Expected Ral,Technical Skills,Standing/Position,Comunication,Maturity,Dynamism,Mobility,English,Hired,Residence Country,Residence Italian Region,Residence Italian Province,Residence Italian City,Italian Residence,European Residence,New_ID
3,114,NaN,NaN,TURIN » Turin ~ Piedmont,Female,False,"-, 3D PRINTING PREFORM SOFTWARE; PYTHON; ANSYS...",Biomedical Engineering,Five-year degree,[0],not specified,not specified,[2022],not specified,not specified,not specified,not specified,cv request,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,NaN,NaN,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,ITALY,TURIN,TURIN,PIEDMONT,True,True,114
4,114,NaN,NaN,TURIN » Turin ~ Piedmont,Female,False,"-, 3D PRINTING PREFORM SOFTWARE; PYTHON; ANSYS...",Biomedical Engineering,Five-year degree,[0],not specified,not specified,[2022],not specified,not specified,not specified,not specified,contact note,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,NaN,NaN,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,ITALY,TURIN,TURIN,PIEDMONT,True,True,114
5,114,NaN,NaN,TURIN » Turin ~ Piedmont,Female,False,"-, 3D PRINTING PREFORM SOFTWARE; PYTHON; ANSYS...",Biomedical Engineering,Five-year degree,[0],not specified,not specified,[2022],not specified,not specified,not specified,not specified,contact note,not specified,~ 2 - Medium,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,NaN,NaN,2.0,2.0,3.0,3.0,3.0,2.0,2.0,not specified,ITALY,TURIN,TURIN,PIEDMONT,True,True,114
6,114,NaN,NaN,TURIN » Turin ~ Piedmont,Female,False,"-, 3D PRINTING PREFORM SOFTWARE; PYTHON; ANSYS...",Biomedical Engineering,Five-year degree,[0],not specified,not specified,[2022],not specified,not specified,not specified,not specified,technical interview,OK,~ 2 - Medium,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,NaN,NaN,2.0,2.0,3.0,3.0,3.0,2.0,2.0,not specified,ITALY,TURIN,TURIN,PIEDMONT,True,True,114
8,137,NaN,NaN,NAPLES » Naples ~ Campania,Female,False,PROJECT MANAGEMENT,Management Engineering,Five-year degree,[0],Others,project manager,[2023],not specified,not specified,not specified,not specified,research association,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,NaN,NaN,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,ITALY,NAPLES,NAPLES,CAMPANIA,True,True,137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9377,80646_1,NaN,NaN,ROME » Rome ~ Lazio,Male,False,not specified,Mechanical engineering,Five-year degree,[+10],Oil and Gas,project manager,[2022],not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,NaN,NaN,not specified,not specified,not specified,not specified,not specified,not specified,not specified,not specified,ITALY,ROME,ROME,LAZIO,True,True,80646_1
9389,81271_2,NaN,NaN,PESCARA » Pescara ~ Abruzzo,Male,False,not specified,

In [89]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from difflib import get_close_matches

# Load raw dataset
df = pd.read_csv('candidates_raw.csv')

### 1. Missing Value Replacement ###
df.fillna('not specified', inplace=True)

### 2. Format numeric fields ###
num_cols = ['Current Ral', 'Expected Ral']
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

### 3. Protected Category → Binary ###
df['Protected Category'] = df['Protected Category'].apply(lambda x: 'yes' if 'article' in str(x).lower() else 'no')

### 4. Normalize Residence ###
def parse_residence(residence):
    # Simulate a geocoding parser: dummy extract country, region, city from string
    parts = residence.split(',')
    country = parts[-1].strip() if len(parts) > 1 else 'not specified'
    city = parts[0].strip() if parts else 'not specified'
    return pd.Series([country, city])

df[['Residence Country', 'Residence City']] = df['Residence'].apply(parse_residence)

# Binary flags
df['Italian Residence'] = df['Residence Country'].apply(lambda x: 1 if x.lower() == 'italy' else 0)
df['European Residence'] = df['Residence Country'].apply(lambda x: 1 if x.lower() in ['italy', 'france', 'germany'] else 0)

### 5. Standardize Last Role ###
# Dummy role mapping (can be extended)
role_mapping = {
    'software dev': 'Software Developer',
    'developer': 'Software Developer',
    'data scientist': 'Data Scientist',
}
df['Last Role'] = df['Last Role'].str.lower().replace(role_mapping)

### 6. Handle Duplicates by ID ###
# Detect conflicting duplicate IDs
grouped = df.groupby('ID')
new_ids = {}
for name, group in grouped:
    if group['Sex'].nunique() > 1:
        for idx, row in group.iterrows():
            new_ids[idx] = f"{row['ID']}_{row['Year of Insertion']}"
    else:
        for idx in group.index:
            new_ids[idx] = row['ID']

df['New_ID'] = df.index.map(new_ids)

### 7. Sorting Categorical Variables ###
# Define custom sort orders
cat_order = {
    'Candidate State': ['imported', 'first contact', 'in selection', 'qm', 'vivier', 'economic proposal', 'hired'],
    'Age Range': ['< 20', '20 - 25', '26 - 30', '31 - 35', '36 - 40', '40 - 45', '> 45'],
    'Years Experience': ['not specified', '[0]', '[0-1]', '[1-3]', '[3-5]', '[5-7]', '[7-10]', '[+10]']
}
for col, order in cat_order.items():
    df[col] = pd.Categorical(df[col], categories=order, ordered=True)

df.sort_values(by=['Candidate State', 'Age Range', 'Years Experience'], inplace=True)

### 8. Filtering Candidates ###
# Remove early stage or ambiguous candidates
invalid_states = ['imported', 'first contact']
df = df[~df['Candidate State'].isin(invalid_states)]

# Filter based on event feedback
invalid_feedback = [
    'OK (other candidate)', 'KO (lost availability)', 'OK (hired)',
    'OK (waiting for departure)', 'KO (opportunity closed)', 'KO (retired)',
    'KO (ral)', 'KO (proposed renunciation)'
]
df = df[~df['Event_Feedback'].isin(invalid_feedback)]

### 9. Collapse Events: Aggregate Interview Scores ###
score_cols = ['Technical Skills', 'Standing/Position', 'Communication', 'Dynamism', 'Mobility', 'English']
df_scores = df.groupby('New_ID')[score_cols].mean().reset_index()
df_final = df.drop_duplicates('New_ID').merge(df_scores, on='New_ID', suffixes=('', '_avg'))

### 10. Final Dataset: Binary Labels (Hired = 1) ###
df_final['Hired'] = (df_final['Candidate State'] == 'hired').astype(int)

### 11. Output Final Dataset ###
df_final.to_csv('processed_candidates.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'candidates_raw.csv'